<a href="https://colab.research.google.com/github/aknanai/scripts/blob/master/ama_divers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing necessary packages



In [ ]:
!pip install yfinance
!pip install stockstats
!pip install pandas

set these variables to what suits you

1.   start_date
2.   end_date
3.   ticker = 'TQQQ" for example

you will get a chart with several kind of moving averages SMA is the best


In [ ]:
import pandas as pd
#from pandas_datareader import data
import numpy as np
import scipy.stats as stats

start_date = '2018-01-01'
end_date = '2023-06-15'
ticker ='TQQQ'
SRC_DATA_FILENAME = 'goog_data.pkl'

from stockstats import StockDataFrame
import yfinance as yf


def sma(prices, periods=200):
    ans = prices.rolling(window=periods, min_periods=periods).mean()
    ans.index.name = 'date'
    ans.columns = 'sma'
    return ans


def rank_last(data):
    s = stats.rankdata(data, method='dense')
    return 1 - s[len(s) - 1] / max(s)


def std_zero(data):
    f = 1.0 / (len(data) - 1)
    return np.sqrt(f * np.power(data, 2).sum())


def ama(prices, rets=None, volatility_periods=25, ranking_periods=2000):
    ama_value = []
    if rets is None:
        rets = prices.pct_changes()
    vola = rets.rolling(window=volatility_periods, min_periods=volatility_periods).apply(std_zero, raw=True).fillna(0)
    ranked_vola = vola.expanding(min_periods=ranking_periods).apply(rank_last, raw=True)

    sc = 0.075
    fc = 0.75

    ans = prices.copy()

    alphas = prices.copy()

    prev = None
    for i in range(len(ranked_vola)):
        if np.isfinite(ranked_vola[i]):
            alpha = np.power(ranked_vola[i] * (fc - sc) + sc, 2)

            alphas[i] = alpha

            alpha = max(alpha, 0.01)
            alpha = min(alpha, 1.0)

            if prev is None:
                prev = prices[i - 1]

            ans[i] = alpha * prices[i] + (1.0 - alpha) * prev

            prev = ans[i]
        else:
            ans[i] = np.nan
            alphas[i] = np.nan

    ans = pd.concat([ans, alphas], axis=1)
    ans.columns = ['ma', 'alpha']
    return ans


def get_stock(ticker='', start='', end=''):
  temp = yf.Ticker(str(ticker))
  Hist_data = temp.history(start=start, end=end)
  df = StockDataFrame.retype(Hist_data)
  df.columns
  #df = df[['open', 'high', 'low', 'close', 'volume']]
  #df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

  return pd.DataFrame(df)

'''
try:
    goog_data2 = pd.read_pickle(SRC_DATA_FILENAME)
except FileNotFoundError:
    goog_data2 = get_stock(ticker='GOOG', start=start_date, end=end_date)
    goog_data2.to_pickle(SRC_DATA_FILENAME)
'''

goog_data2 = get_stock(ticker=ticker, start=start_date, end=end_date)
#goog_data = goog_data2.tail(620)
goog_data = goog_data2

close = goog_data['close']
RET = close.pct_change()
low = goog_data['low']
AMA = ama(goog_data.low, RET, ranking_periods=250, volatility_periods=3)

'''
The Absolute Price Oscillator (APO) is based
 on the absolute differences between two moving averages of different
 lengths, a ‘Fast’ and a ‘Slow’ moving average.

APO = Fast Exponential Moving Average - Slow Exponential Moving Average
'''
num_periods_fast = 10  # time period for the fast EMA
K_fast = 2 / (num_periods_fast + 1)  # smoothing factor for fast EMA
ema_fast = 0
num_periods_slow = 40  # time period for slow EMA
K_slow = 2 / (num_periods_slow + 1)  # smoothing factor for slow EMA
ema_slow = 0

ema_fast_values = []  # we will hold fast EMA values for visualization purposes
ema_slow_values = []  # we will hold slow EMA values for visualization purposes
apo_values = []  # track computed absolute price oscillator values
ama_value = []
for (close_price, low_price) in zip(close, low):
    if ema_fast == 0:  # first observation
        ema_fast = low_price
        ema_slow = close_price
    else:
        ema_fast = (low_price - ema_fast) * K_fast + ema_fast
        ema_slow = (close_price - ema_slow) * K_slow + ema_slow

    ema_fast_values.append(ema_fast)
    ema_slow_values.append(ema_slow)
    apo_values.append(ema_fast - ema_slow)

goog_data = goog_data.assign(ClosePrice=pd.Series(close, index=goog_data.index))
goog_data = goog_data.assign(FastExponential10DayMovingAverage=pd.Series(ema_fast_values, index=goog_data.index))
goog_data = goog_data.assign(SlowExponential40DayMovingAverage=pd.Series(ema_slow_values, index=goog_data.index))
goog_data = goog_data.assign(AbsolutePriceOscillator=pd.Series(apo_values, index=goog_data.index))
goog_data["AMA-CLOSE"] = AMA['ma']
# goog_data=goog_data.assign("AMANOU")=pd.Series()


close_price = goog_data['ClosePrice']
ema_f = goog_data['FastExponential10DayMovingAverage']
ema_s = goog_data['SlowExponential40DayMovingAverage']
apo = goog_data['AbsolutePriceOscillator']
ama_test = goog_data['AMA-CLOSE']

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(14, 18))
ax1 = fig.add_subplot(211, ylabel='Google price in $')
close_price.plot(ax=ax1, color='g', lw=2., legend=True)
ema_f.plot(ax=ax1, color='b', lw=2., legend=True)
ema_s.plot(ax=ax1, color='r', lw=2., legend=True)
ama_test.plot(ax=ax1, color='brown', lw=2, legend=True)
ax2 = fig.add_subplot(212, ylabel='APO')
apo.plot(ax=ax2, color='black', lw=2., legend=True)
plt.show()

print_ama = True
if print_ama:
    fig = plt.figure(figsize=(14, 18))
    ax1 = fig.add_subplot(211, ylabel='Google price in $')
    close_price.plot(ax=ax1, color='g', lw=2., legend=True)
    ama_test.plot(ax=ax1, color='b', lw=2, legend=True)
    plt.show()


You can use this code to find the saisonability of a ticker during the periode of time you provided ( start and end time above)

In [ ]:
goog_data2 = get_stock(ticker='Aapl', start=start_date, end=end_date)
#goog_data = goog_data2.tail(620)
goog_data = goog_data2


goog_monthly_return=goog_data['close'].pct_change().groupby([goog_data['close'].index.year,goog_data['close'].index.month]).mean()

goog_monthly_return_list=[]

for i in  range(len(goog_monthly_return)):
  goog_monthly_return_list.append({'month':goog_monthly_return.index[i][1],'monthly_return':goog_monthly_return.iloc[i]})

goog_monthly_return_list=pd.DataFrame(goog_monthly_return_list,columns=('month','monthly_return'))
goog_monthly_return_list
goog_monthly_return


goog_monthly_return_list.boxplot(column='monthly_return',by='month')


In [ ]:
goog_data2 = get_stock(ticker='goog', start=start_date, end=end_date)
#goog_data = goog_data2.tail(620)
goog_data = goog_data2
#data = yf.download(tickers="tqqq", period="7d", interval="1m")
#goog_data=data

#goog_weekly_return=goog_data2.groupby(pd.Grouper(freq='1d', base=10, label='right')).first()
goog_weekly_return=goog_data['Close'].pct_change().pd.groupby([goog_data['Close'].index.year,goog_data['Close'].index.hour]).mean()

goog_weekly_return_list=[]

for i in  range(len(goog_weekly_return)):
  goog_weekly_return_list.append({'month':goog_weekly_return.index[i][1],'monthly_return':goog_weekly_return.iloc[i]})

goog_weekly_return_list=pd.DataFrame(goog_weekly_return_list,columns=('hour','weekly_return'))
goog_weekly_return_list
goog_weekly_return
print(goog_weekly_return)

#goog_weekly_return_list.boxplot(column='weekly_return',by='hour')

This will define a lot of tickers symboles few are no more valide, you can skip this code

In [ ]:
tickers = ['A', 'AA', 'AAAU', 'AACG', 'AACQ', 'AADR', 'AAIC', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP',
           'AAPL', 'AAT', 'AAU', 'AAWW', 'AAXJ', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABEQ',
           'ABEV', 'ABG', 'ABIO', 'ABM', 'ABMD', 'ABNB', 'ABR', 'ABST', 'ABT', 'ABTX', 'ABUS', 'AC', 'ACA', 'ACACU',
           'ACAD', 'ACB', 'ACBI', 'ACC', 'ACCD', 'ACCO', 'ACEL', 'ACER', 'ACES', 'ACET', 'ACEV', 'ACEVU', 'ACGL', 'ACH',
           'ACHC', 'ACHV', 'ACI', 'ACIA', 'ACIC', 'ACIO', 'ACIU', 'ACIW', 'ACKIT', 'ACKIU', 'ACLS', 'ACM', 'ACMR',
           'ACN', 'ACNB', 'ACOR', 'ACP', 'ACRE', 'ACRS', 'ACRX', 'ACST', 'ACTC', 'ACTCU', 'ACTG', 'ACU', 'ACV', 'ACWF',
           'ACWI', 'ACWV', 'ACWX', 'ACY', 'ADAP', 'ADBE', 'ADC', 'ADCT', 'ADERU', 'ADES', 'ADI', 'ADIL', 'ADM', 'ADMA',
           'ADME', 'ADMP', 'ADMS', 'ADNT', 'ADOC', 'ADP', 'ADPT', 'ADRE', 'ADS', 'ADSK', 'ADT', 'ADTN', 'ADTX', 'ADUS',
           'ADV', 'ADVM', 'ADX', 'ADXN', 'ADXS', 'AE', 'AEE', 'AEF', 'AEG', 'AEGN', 'AEHL', 'AEHR', 'AEIS', 'AEL',
           'AEM', 'AEMD', 'AEO', 'AEP', 'AER', 'AERI', 'AES', 'AESE', 'AESR', 'AEY', 'AEYE', 'AEZS', 'AFB', 'AFBI',
           'AFG', 'AFI', 'AFIB', 'AFIF', 'AFIN', 'AFINO', 'AFK', 'AFL', 'AFLG', 'AFMD', 'AFRM', 'AFT', 'AFTY', 'AFYA',
           'AG', 'AGCB', 'AGCO', 'AGCUU', 'AGD', 'AGE', 'AGEN', 'AGFS', 'AGFY', 'AGG', 'AGGP', 'AGGY', 'AGI', 'AGIO',
           'AGLE', 'AGM', 'AGMH', 'AGNC', 'AGO', 'AGQ', 'AGR', 'AGRO', 'AGRX', 'AGS', 'AGT', 'AGTC', 'AGX', 'AGYS',
           'AGZ', 'AGZD', 'AHACU', 'AHC', 'AHCO', 'AHH', 'AHPI', 'AHT', 'AI', 'AIA', 'AIEQ', 'AIF', 'AIG', 'AIH',
           'AIHS', 'AIKI', 'AIM', 'AIMC', 'AIN', 'AINC', 'AINV', 'AIO', 'AIQ', 'AIR', 'AIRC', 'AIRG', 'AIRI', 'AIRR',
           'AIRT', 'AIT', 'AIV', 'AIZ', 'AJAX', 'AJG', 'AJRD', 'AJX', 'AKAM', 'AKBA', 'AKER', 'AKO-A', 'AKO-B', 'AKR',
           'AKRO', 'AKTS', 'AKTX', 'AKUS', 'AL', 'ALAC', 'ALB', 'ALBO', 'ALC', 'ALCO', 'ALDX', 'ALE', 'ALEC', 'ALEX',
           'ALFA', 'ALG', 'ALGM', 'ALGN', 'ALGS', 'ALGT', 'ALIM', 'ALJJ', 'ALK', 'ALKS', 'ALL', 'ALLE', 'ALLK', 'ALLO',
           'ALLT', 'ALLY', 'ALNA', 'ALNY', 'ALOT', 'ALPN', 'ALRM', 'ALRN', 'ALRS', 'ALSK', 'ALSN', 'ALT', 'ALTA',
           'ALTG', 'ALTL', 'ALTM', 'ALTO', 'ALTR', 'ALTU', 'ALTUU', 'ALTY', 'ALUS', 'ALV', 'ALVR', 'ALX', 'ALXN',
           'ALXO', 'ALYA', 'AM', 'AMAL', 'AMAT', 'AMBA', 'AMBC', 'AMBO', 'AMC', 'AMCI', 'AMCR', 'AMCX', 'AMD', 'AME',
           'AMED', 'AMEH', 'AMG', 'AMGN', 'AMH', 'AMJ', 'AMK', 'AMKR', 'AMLP', 'AMN', 'AMNB', 'AMOT', 'AMP', 'AMPE',
           'AMPH', 'AMPY', 'AMR', 'AMRB', 'AMRC', 'AMRK', 'AMRN', 'AMRS', 'AMRX', 'AMS', 'AMSC', 'AMSF', 'AMST',
           'AMSWA', 'AMT', 'AMTB', 'AMTI', 'AMTR', 'AMTX', 'AMUB', 'AMWD', 'AMWL', 'AMX', 'AMYT', 'AMZA', 'AMZN', 'AN',
           'ANAB', 'ANAT', 'ANCN', 'ANDA', 'ANDE', 'ANET', 'ANF', 'ANGI', 'ANGL', 'ANGN', 'ANGO', 'ANH', 'ANIK', 'ANIP',
           'ANIX', 'ANNX', 'ANPC', 'ANSS', 'ANTE', 'ANTM', 'ANVS', 'ANY', 'AOA', 'AOD', 'AOK', 'AOM', 'AON', 'AOR',
           'AOS', 'AOSL', 'AOUT', 'AP', 'APA', 'APAM', 'APD', 'APDN', 'APEI', 'APEN', 'APG', 'APH', 'APHA', 'API',
           'APLE', 'APLS', 'APLT', 'APM', 'APO', 'APOG', 'APOP', 'APPF', 'APPH', 'APPN', 'APPS', 'APRE', 'APRN', 'APSG',
           'APT', 'APTO', 'APTS', 'APTV', 'APTX', 'APVO', 'APWC', 'APXT', 'APYX', 'AQB', 'AQMS', 'AQN', 'AQST', 'AQUA',
           'AR', 'ARAV', 'ARAY', 'ARBGU', 'ARC', 'ARCB', 'ARCC', 'ARCE', 'ARCH', 'ARCO', 'ARCT', 'ARD', 'ARDC', 'ARDS',
           'ARDX', 'ARE', 'AREC', 'ARES', 'ARGO', 'ARGT', 'ARGX', 'ARI', 'ARKF', 'ARKG', 'ARKIU', 'ARKK', 'ARKO',
           'ARKQ', 'ARKR', 'ARKW', 'ARL', 'ARLO', 'ARLP', 'ARMK', 'ARMP', 'ARMR', 'ARNA', 'ARNC', 'AROC', 'AROW',
           'ARPO', 'ARQT', 'ARR', 'ARRY', 'ARTL', 'ARTNA', 'ARTW', 'ARVN', 'ARW', 'ARWR', 'ARYA', 'ASA', 'ASAN',
           'ASAXU', 'ASB', 'ASC', 'ASEA', 'ASET', 'ASG', 'ASGI', 'ASGN', 'ASH', 'ASHR', 'ASHS', 'ASIX', 'ASLE', 'ASLN',
           'ASM', 'ASMB', 'ASML', 'ASND', 'ASO', 'ASPN', 'ASPS', 'ASPU', 'ASR', 'ASRT', 'ASRV', 'ASTC', 'ASTE', 'ASUR',
           'ASX', 'ASYS', 'AT', 'ATAC', 'ATAX', 'ATC', 'ATCO', 'ATCX', 'ATEC', 'ATEN', 'ATEX', 'ATGE', 'ATH', 'ATHA',
           'ATHE', 'ATHM', 'ATHX', 'ATI', 'ATIF', 'ATKR', 'ATLC', 'ATLO', 'ATMP', 'ATNF', 'ATNI', 'ATNM', 'ATNX', 'ATO',
           'ATOM', 'ATOS', 'ATR', 'ATRA', 'ATRC', 'ATRI', 'ATRO', 'ATRS', 'ATSG', 'ATTO', 'ATUS', 'ATVI', 'ATXI', 'AU',
           'AUB', 'AUBN', 'AUDC', 'AUMN', 'AUPH', 'AUSF', 'AUTL', 'AUTO', 'AUVI', 'AUY', 'AVA', 'AVAL', 'AVAN', 'AVAV',
           'AVB', 'AVCO', 'AVCT', 'AVD', 'AVDE', 'AVDL', 'AVDV', 'AVEM', 'AVEO', 'AVGO', 'AVGR', 'AVID', 'AVIR', 'AVK',
           'AVLR', 'AVNS', 'AVNT', 'AVNW', 'AVO', 'AVRO', 'AVT', 'AVTR', 'AVUS', 'AVUV', 'AVXL', 'AVY', 'AVYA', 'AWAY',
           'AWF', 'AWH', 'AWI', 'AWK', 'AWP', 'AWR', 'AWRE', 'AWTM', 'AWX', 'AX', 'AXAS', 'AXDX', 'AXGN', 'AXL', 'AXLA',
           'AXNX', 'AXON', 'AXP', 'AXR', 'AXS', 'AXSM', 'AXTA', 'AXTI', 'AXU', 'AY', 'AYI', 'AYLA', 'AYRO', 'AYTU',
           'AYX', 'AZEK', 'AZN', 'AZO', 'AZPN', 'AZRE', 'AZRX', 'AZUL', 'AZYO', 'AZZ', 'B', 'BA', 'BAB', 'BABA', 'BAC',
           'BAF', 'BAH', 'BAK', 'BAL', 'BALY', 'BAM', 'BANC', 'BAND', 'BANF', 'BANR', 'BANX', 'BAP', 'BAPR', 'BAR',
           'BASI', 'BATL', 'BATRA', 'BATRK', 'BATT', 'BAUG', 'BAX', 'BB', 'BBAR', 'BBAX', 'BBBY', 'BBC', 'BBCA', 'BBCP',
           'BBD', 'BBDC', 'BBEU', 'BBF', 'BBGI', 'BBH', 'BBI', 'BBIG', 'BBIN', 'BBIO', 'BBJP', 'BBK', 'BBL', 'BBMC',
           'BBN', 'BBQ', 'BBRE', 'BBSA', 'BBSI', 'BBU', 'BBUS', 'BBVA', 'BBW', 'BBY', 'BC', 'BCAB', 'BCACU', 'BCBP',
           'BCC', 'BCD', 'BCDA', 'BCE', 'BCEI', 'BCEL', 'BCH', 'BCI', 'BCLI', 'BCM', 'BCML', 'BCO', 'BCOR', 'BCOV',
           'BCPC', 'BCRX', 'BCS', 'BCSF', 'BCTG', 'BCV', 'BCX', 'BCYC', 'BDC', 'BDCZ', 'BDEC', 'BDJ', 'BDL', 'BDN',
           'BDR', 'BDRY', 'BDSI', 'BDSX', 'BDTX', 'BDX', 'BE', 'BEAM', 'BEAT', 'BECN', 'BEDU', 'BEEM', 'BEKE', 'BELFB',
           'BEN', 'BENE', 'BENEU', 'BEP', 'BEPC', 'BERY', 'BEST', 'BETZ', 'BF-A', 'BF-B', 'BFAM', 'BFEB', 'BFI', 'BFIN',
           'BFIT', 'BFK', 'BFOR', 'BFRA', 'BFS', 'BFST', 'BFT', 'BFY', 'BFZ', 'BG', 'BGB', 'BGCP', 'BGFV', 'BGH', 'BGI',
           'BGIO', 'BGNE', 'BGR', 'BGRN', 'BGS', 'BGSF', 'BGT', 'BGX', 'BGY', 'BH', 'BHAT', 'BHB', 'BHC', 'BHE', 'BHF',
           'BHFAN', 'BHK', 'BHLB', 'BHP', 'BHR', 'BHSEU', 'BHTG', 'BHV', 'BHVN', 'BIB', 'BIBL', 'BICK', 'BIDU', 'BIF',
           'BIG', 'BIGC', 'BIIB', 'BIL', 'BILI', 'BILL', 'BIMI', 'BIO', 'BIOC', 'BIOL', 'BIOX', 'BIP', 'BIPC', 'BIS',
           'BIT', 'BIV', 'BIVI', 'BIZD', 'BJ', 'BJAN', 'BJK', 'BJRI', 'BJUL', 'BJUN', 'BK', 'BKCC', 'BKD', 'BKE',
           'BKEP', 'BKF', 'BKH', 'BKI', 'BKLC', 'BKLN', 'BKN', 'BKNG', 'BKR', 'BKSC', 'BKSE', 'BKT', 'BKTI', 'BKU',
           'BKYI', 'BL', 'BLBD', 'BLCM', 'BLCN', 'BLCT', 'BLD', 'BLDP', 'BLDR', 'BLE', 'BLES', 'BLFS', 'BLI', 'BLIN',
           'BLK', 'BLKB', 'BLL', 'BLMN', 'BLNK', 'BLOK', 'BLPH', 'BLRX', 'BLSA', 'BLTSU', 'BLU', 'BLUE', 'BLUWU', 'BLV',
           'BLW', 'BLX', 'BMA', 'BMAR', 'BMAY', 'BME', 'BMEZ', 'BMI', 'BMO', 'BMRA', 'BMRC', 'BMRN', 'BMTC', 'BMTX',
           'BMY', 'BND', 'BNDC', 'BNDW', 'BNDX', 'BNED', 'BNFT', 'BNGO', 'BNKD', 'BNKU', 'BNL', 'BNO', 'BNOV', 'BNR',
           'BNS', 'BNSO', 'BNTC', 'BNTX', 'BNY', 'BOAC', 'BOCH', 'BOCT', 'BOE', 'BOH', 'BOIL', 'BOKF', 'BOLT', 'BOMN',
           'BOND', 'BOOM', 'BOOT', 'BORR', 'BOSC', 'BOTJ', 'BOTZ', 'BOUT', 'BOWXU', 'BOX', 'BOXL', 'BP', 'BPFH', 'BPMC',
           'BPMP', 'BPOP', 'BPT', 'BPTH', 'BPY', 'BPYU', 'BQ', 'BR', 'BRBR', 'BRC', 'BREZ', 'BRF', 'BRFS', 'BRG',
           'BRID', 'BRK-A', 'BRK-B', 'BRKL', 'BRKR', 'BRKS', 'BRLI', 'BRLIU', 'BRMK', 'BRN', 'BRO', 'BROG', 'BRP',
           'BRPA', 'BRQS', 'BRT', 'BRX', 'BRY', 'BRZU', 'BSA', 'BSAC', 'BSAE', 'BSBK', 'BSBR', 'BSCL', 'BSCM', 'BSCN',
           'BSCO', 'BSCP', 'BSCQ', 'BSCR', 'BSCS', 'BSCT', 'BSD', 'BSDE', 'BSE', 'BSEP', 'BSET', 'BSGM', 'BSIG', 'BSJL',
           'BSJM', 'BSJN', 'BSJO', 'BSJP', 'BSJQ', 'BSJR', 'BSL', 'BSM', 'BSML', 'BSMM', 'BSMN', 'BSMO', 'BSMP', 'BSMQ',
           'BSMR', 'BSMS', 'BSMX', 'BSQR', 'BSRR', 'BST', 'BSTZ', 'BSV', 'BSX', 'BSY', 'BTA', 'BTAI', 'BTAL', 'BTAQ',
           'BTBT', 'BTEC', 'BTEK', 'BTG', 'BTI', 'BTN', 'BTNB', 'BTO', 'BTRS', 'BTT', 'BTU', 'BTWN', 'BTWNU', 'BTZ',
           'BUD', 'BUG', 'BUI', 'BUR', 'BURL', 'BUSE', 'BUYZ', 'BV', 'BVH', 'BVN', 'BVXV', 'BW', 'BWA', 'BWAC', 'BWACU',
           'BWAY', 'BWB', 'BWEN', 'BWFG', 'BWG', 'BWL-A', 'BWMX', 'BWX', 'BWXT', 'BWZ', 'BX', 'BXC', 'BXG', 'BXMT',
           'BXMX', 'BXP', 'BXRX', 'BXS', 'BY', 'BYD', 'BYFC', 'BYLD', 'BYM', 'BYND', 'BYSI', 'BZH', 'BZM', 'BZQ',
           'BZUN', 'C', 'CAAP', 'CAAS', 'CABA', 'CABO', 'CAC', 'CACC', 'CACG', 'CACI', 'CADE', 'CAE', 'CAF', 'CAG',
           'CAH', 'CAHCU', 'CAI', 'CAJ', 'CAKE', 'CAL', 'CALA', 'CALF', 'CALM', 'CALT', 'CALX', 'CAMP', 'CAMT', 'CAN',
           'CANE', 'CANF', 'CANG', 'CAP', 'CAPE', 'CAPL', 'CAPR', 'CAR', 'CARA', 'CARE', 'CARG', 'CARR', 'CARS', 'CARV',
           'CARZ', 'CAS', 'CASA', 'CASH', 'CASI', 'CASS', 'CASY', 'CAT', 'CATB', 'CATC', 'CATH', 'CATM', 'CATO', 'CATY',
           'CB', 'CBAH', 'CBAN', 'CBAT', 'CBAY', 'CBB', 'CBD', 'CBFV', 'CBH', 'CBIO', 'CBLI', 'CBLS', 'CBMB', 'CBMG',
           'CBNK', 'CBOE', 'CBON', 'CBPO', 'CBRE', 'CBRL', 'CBSH', 'CBT', 'CBTG', 'CBTX', 'CBU', 'CBZ', 'CC', 'CCAP',
           'CCBG', 'CCCC', 'CCD', 'CCEP', 'CCF', 'CCI', 'CCIV', 'CCJ', 'CCK', 'CCL', 'CCLP', 'CCM', 'CCMP', 'CCNC',
           'CCNE', 'CCO', 'CCOI', 'CCOR', 'CCRC', 'CCRN', 'CCS', 'CCU', 'CCV', 'CCX', 'CCXI', 'CD', 'CDAK', 'CDAY',
           'CDC', 'CDE', 'CDEV', 'CDK', 'CDL', 'CDLX', 'CDMO', 'CDNA', 'CDNS', 'CDOR', 'CDR', 'CDTX', 'CDW', 'CDXC',
           'CDXS', 'CDZI', 'CE', 'CEA', 'CECE', 'CEE', 'CEF', 'CEFS', 'CEI', 'CEIX', 'CEL', 'CELC', 'CELH', 'CELP',
           'CEM', 'CEMB', 'CEMI', 'CEN', 'CENHU', 'CENT', 'CENTA', 'CENX', 'CEO', 'CEPU', 'CEQP', 'CERC', 'CERE',
           'CERN', 'CERS', 'CERT', 'CET', 'CETX', 'CEV', 'CEVA', 'CEW', 'CEY', 'CEZ', 'CF', 'CFA', 'CFAC', 'CFACU',
           'CFB', 'CFBK', 'CFFI', 'CFFN', 'CFFVU', 'CFG', 'CFIIU', 'CFIVU', 'CFMS', 'CFO', 'CFR', 'CFRX', 'CFX', 'CG',
           'CGA', 'CGBD', 'CGC', 'CGEM', 'CGEN', 'CGIX', 'CGNT', 'CGNX', 'CGO', 'CGRO', 'CGROU', 'CGW', 'CHAD', 'CHAU',
           'CHCI', 'CHCO', 'CHCT', 'CHD', 'CHDN', 'CHE', 'CHEF', 'CHEK', 'CHFS', 'CHFW', 'CHGG', 'CHGX', 'CHH', 'CHI',
           'CHIC', 'CHIH', 'CHIK', 'CHIL', 'CHIQ', 'CHIR', 'CHIS', 'CHIX', 'CHKP', 'CHMA', 'CHMG', 'CHMI', 'CHN',
           'CHNA', 'CHNG', 'CHNR', 'CHPM', 'CHRA', 'CHRS', 'CHRW', 'CHS', 'CHSCL', 'CHSCM', 'CHSCN', 'CHSCO', 'CHSCP',
           'CHT', 'CHTR', 'CHUY', 'CHW', 'CHWY', 'CHX', 'CHY', 'CI', 'CIA', 'CIB', 'CIBR', 'CID', 'CIDM', 'CIEN', 'CIF',
           'CIG', 'CIGI', 'CIH', 'CII', 'CIIC', 'CIK', 'CIL', 'CIM', 'CINF', 'CINR', 'CIO', 'CIR', 'CIT', 'CIVB', 'CIX',
           'CIXX', 'CIZ', 'CIZN', 'CJJD', 'CKH', 'CKPT', 'CKX', 'CL', 'CLA', 'CLAR', 'CLB', 'CLBK', 'CLBS', 'CLCT',
           'CLDR', 'CLDT', 'CLDX', 'CLEU', 'CLF', 'CLFD', 'CLGN', 'CLGX', 'CLH', 'CLI', 'CLII', 'CLIR', 'CLIX', 'CLLS',
           'CLM', 'CLMT', 'CLNC', 'CLNE', 'CLNN', 'CLNY', 'CLOU', 'CLOV', 'CLPR', 'CLPS', 'CLPT', 'CLR', 'CLRB', 'CLRO',
           'CLS', 'CLSD', 'CLSK', 'CLSN', 'CLTL', 'CLVR', 'CLVS', 'CLVT', 'CLW', 'CLWT', 'CLX', 'CLXT', 'CM', 'CMA',
           'CMBM', 'CMBS', 'CMC', 'CMCL', 'CMCM', 'CMCO', 'CMCSA', 'CMCT', 'CMD', 'CMDY', 'CME', 'CMF', 'CMG', 'CMI',
           'CMLFU', 'CMLS', 'CMO', 'CMP', 'CMPI', 'CMPR', 'CMPS', 'CMRE', 'CMRX', 'CMS', 'CMT', 'CMTL', 'CMU', 'CN',
           'CNA', 'CNBKA', 'CNBS', 'CNC', 'CNCE', 'CNCR', 'CND', 'CNDT', 'CNET', 'CNEY', 'CNF', 'CNFR', 'CNHI', 'CNI',
           'CNK', 'CNMD', 'CNNE', 'CNO', 'CNOB', 'CNP', 'CNQ', 'CNR', 'CNRG', 'CNS', 'CNSL', 'CNSP', 'CNST', 'CNTG',
           'CNTY', 'CNX', 'CNXC', 'CNXN', 'CNXT', 'CNYA', 'CO', 'COCP', 'CODA', 'CODI', 'CODX', 'COE', 'COF', 'COG',
           'COGT', 'COHN', 'COHR', 'COHU', 'COKE', 'COLB', 'COLD', 'COLL', 'COLM', 'COM', 'COMB', 'COMM', 'COMS',
           'COMT', 'CONE', 'CONN', 'CONX', 'CONXU', 'COO', 'COOP', 'COP', 'COPX', 'COR', 'CORE', 'CORN', 'CORP', 'CORR',
           'CORT', 'COST', 'COTY', 'COUP', 'COW', 'COWN', 'COWZ', 'CP', 'CPA', 'CPAC', 'CPAH', 'CPB', 'CPE', 'CPER',
           'CPF', 'CPG', 'CPHC', 'CPHI', 'CPI', 'CPIX', 'CPK', 'CPLG', 'CPLP', 'CPRI', 'CPRT', 'CPRX', 'CPS', 'CPSH',
           'CPSI', 'CPSS', 'CPST', 'CPT', 'CPTA', 'CQP', 'CQQQ', 'CR', 'CRAI', 'CRAK', 'CRBN', 'CRBP', 'CRC', 'CRD-A',
           'CRD-B', 'CRDF', 'CREE', 'CREG', 'CRESY', 'CREX', 'CRF', 'CRH', 'CRHC', 'CRHM', 'CRI', 'CRIS', 'CRK', 'CRKN',
           'CRL', 'CRM', 'CRMD', 'CRMT', 'CRNC', 'CRNT', 'CRNX', 'CROC', 'CRON', 'CROX', 'CRS', 'CRSA', 'CRSP', 'CRSR',
           'CRT', 'CRTD', 'CRTO', 'CRTX', 'CRUS', 'CRVL', 'CRVS', 'CRWD', 'CRWS', 'CRY', 'CS', 'CSB', 'CSBR', 'CSCO',
           'CSCW', 'CSD', 'CSF', 'CSGP', 'CSGS', 'CSII', 'CSIQ', 'CSL', 'CSLT', 'CSM', 'CSML', 'CSOD', 'CSPI', 'CSPR',
           'CSQ', 'CSR', 'CSSE', 'CSTE', 'CSTL', 'CSTM', 'CSTR', 'CSU', 'CSV', 'CSWC', 'CSWI', 'CSX', 'CTAC', 'CTAQ',
           'CTAQU', 'CTAS', 'CTB', 'CTBB', 'CTBI', 'CTEC', 'CTEK', 'CTG', 'CTHR', 'CTIB', 'CTIC', 'CTK', 'CTLT', 'CTMX',
           'CTO', 'CTR', 'CTRA', 'CTRE', 'CTRM', 'CTRN', 'CTS', 'CTSH', 'CTSO', 'CTT', 'CTVA', 'CTXR', 'CTXS', 'CUB',
           'CUBA', 'CUBE', 'CUBI', 'CUE', 'CUK', 'CULP', 'CURE', 'CURI', 'CURO', 'CUT', 'CUTR', 'CUZ', 'CVA', 'CVAC',
           'CVBF', 'CVCO', 'CVCY', 'CVE', 'CVEO', 'CVET', 'CVGI', 'CVGW', 'CVI', 'CVLB', 'CVLG', 'CVLT', 'CVLY', 'CVM',
           'CVNA', 'CVR', 'CVS', 'CVU', 'CVV', 'CVX', 'CVY', 'CW', 'CWB', 'CWBC', 'CWBR', 'CWCO', 'CWEB', 'CWEN',
           'CWEN-A', 'CWH', 'CWI', 'CWK', 'CWS', 'CWST', 'CWT', 'CX', 'CXDC', 'CXDO', 'CXE', 'CXH', 'CXP', 'CXSE',
           'CXW', 'CYAD', 'CYAN', 'CYB', 'CYBE', 'CYBR', 'CYCC', 'CYCN', 'CYD', 'CYH', 'CYRN', 'CYRX', 'CYTH', 'CYTK',
           'CZA', 'CZNC', 'CZR', 'CZWI', 'CZZ', 'D', 'DAC', 'DADA', 'DAIO', 'DAKT', 'DAL', 'DALI', 'DALT', 'DAN', 'DAO',
           'DAR', 'DARE', 'DASH', 'DAUG', 'DAVA', 'DAX', 'DB', 'DBA', 'DBAW', 'DBB', 'DBC', 'DBD', 'DBDR', 'DBE',
           'DBEF', 'DBEM', 'DBEU', 'DBEZ', 'DBI', 'DBJA', 'DBJP', 'DBL', 'DBMF', 'DBO', 'DBP', 'DBS', 'DBVT', 'DBX',
           'DCBO', 'DCI', 'DCO', 'DCOM', 'DCP', 'DCPH', 'DCRB', 'DCRBU', 'DCRNU', 'DCT', 'DCTH', 'DD', 'DDD', 'DDEC',
           'DDF', 'DDG', 'DDIV', 'DDLS', 'DDM', 'DDMX', 'DDMXU', 'DDOG', 'DDS', 'DDWM', 'DE', 'DEA', 'DECK', 'DEEF',
           'DEEP', 'DEF', 'DEH', 'DEI', 'DELL', 'DEM', 'DEMZ', 'DEN', 'DENN', 'DEO', 'DES', 'DESP', 'DEUS', 'DEW',
           'DEX', 'DFAI', 'DFAU', 'DFE', 'DFEB', 'DFEN', 'DFFN', 'DFH', 'DFIN', 'DFJ', 'DFND', 'DFNL', 'DFP', 'DFS',
           'DG', 'DGICA', 'DGII', 'DGL', 'DGLY', 'DGNR', 'DGNS', 'DGP', 'DGRE', 'DGRO', 'DGRS', 'DGRW', 'DGS', 'DGT',
           'DGX', 'DGZ', 'DHC', 'DHF', 'DHI', 'DHIL', 'DHR', 'DHS', 'DHT', 'DHX', 'DHY', 'DIA', 'DIAL', 'DIAX', 'DIG',
           'DIM', 'DIN', 'DINT', 'DIOD', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DIT', 'DIV', 'DIVB', 'DIVO', 'DJAN', 'DJCO',
           'DJD', 'DJP', 'DJUN', 'DK', 'DKL', 'DKNG', 'DKS', 'DL', 'DLA', 'DLB', 'DLCAU', 'DLHC', 'DLN', 'DLNG', 'DLPN',
           'DLR', 'DLS', 'DLTH', 'DLTR', 'DLX', 'DM', 'DMAC', 'DMB', 'DMF', 'DMLP', 'DMO', 'DMRC', 'DMRL', 'DMS',
           'DMTK', 'DMYD', 'DMYI', 'DNB', 'DNK', 'DNL', 'DNLI', 'DNMR', 'DNN', 'DNOV', 'DNOW', 'DNP', 'DOC', 'DOCU',
           'DOG', 'DOGZ', 'DOL', 'DOMO', 'DON', 'DOO', 'DOOO', 'DOOR', 'DORM', 'DOV', 'DOW', 'DOX', 'DOYU', 'DPG',
           'DPST', 'DPW', 'DPZ', 'DQ', 'DRD', 'DRE', 'DRH', 'DRI', 'DRIO', 'DRIP', 'DRIV', 'DRN', 'DRNA', 'DRQ', 'DRRX',
           'DRSK', 'DRTT', 'DRV', 'DRVN', 'DRW', 'DS', 'DSAC', 'DSACU', 'DSE', 'DSGX', 'DSI', 'DSKE', 'DSL', 'DSM',
           'DSPG', 'DSS', 'DSSI', 'DSTL', 'DSU', 'DSWL', 'DSX', 'DT', 'DTD', 'DTE', 'DTEA', 'DTEC', 'DTF', 'DTH',
           'DTIL', 'DTJ', 'DTN', 'DTSS', 'DUC', 'DUDE', 'DUG', 'DUK', 'DUKH', 'DUNEU', 'DUO', 'DUOT', 'DURA', 'DUSA',
           'DUSL', 'DUST', 'DVA', 'DVAX', 'DVD', 'DVLU', 'DVN', 'DVOL', 'DVY', 'DVYA', 'DVYE', 'DWAS', 'DWAW', 'DWEQ',
           'DWFI', 'DWIN', 'DWLD', 'DWM', 'DWPP', 'DWSH', 'DWSN', 'DWUS', 'DWX', 'DX', 'DXC', 'DXCM', 'DXD', 'DXF',
           'DXJ', 'DXJS', 'DXPE', 'DXR', 'DXYN', 'DY', 'DYAI', 'DYN', 'DYNF', 'DYNT', 'DZSI', 'DZZ', 'E', 'EA', 'EACPU',
           'EAD', 'EAF', 'EAGG', 'EAR', 'EARN', 'EARS', 'EASG', 'EAST', 'EAT', 'EB', 'EBAY', 'EBC', 'EBF', 'EBIX',
           'EBIZ', 'EBMT', 'EBND', 'EBON', 'EBR', 'EBR-B', 'EBS', 'EBSB', 'EBTC', 'EC', 'ECC', 'ECF', 'ECH', 'ECHO',
           'ECL', 'ECLN', 'ECNS', 'ECOL', 'ECOM', 'ECON', 'ECOR', 'ECPG', 'ED', 'EDAP', 'EDC', 'EDD', 'EDEN', 'EDF',
           'EDI', 'EDIT', 'EDIV', 'EDN', 'EDOC', 'EDOG', 'EDOW', 'EDRY', 'EDSA', 'EDTK', 'EDTXU', 'EDU', 'EDUC', 'EDUT',
           'EDV', 'EDZ', 'EEA', 'EEFT', 'EELV', 'EEM', 'EEMA', 'EEMD', 'EEMO', 'EEMS', 'EEMV', 'EEMX', 'EES', 'EET',
           'EEV', 'EEX', 'EFA', 'EFAD', 'EFAS', 'EFAV', 'EFAX', 'EFC', 'EFF', 'EFG', 'EFNL', 'EFO', 'EFOI', 'EFR',
           'EFSC', 'EFT', 'EFV', 'EFX', 'EFZ', 'EGAN', 'EGBN', 'EGF', 'EGHT', 'EGLE', 'EGO', 'EGOV', 'EGP', 'EGPT',
           'EGRX', 'EGY', 'EH', 'EHC', 'EHI', 'EHT', 'EHTH', 'EIDO', 'EIG', 'EIGI', 'EIGR', 'EIM', 'EINC', 'EIRL',
           'EIS', 'EIX', 'EJAN', 'EJUL', 'EKAR', 'EKSO', 'EL', 'ELA', 'ELAN', 'ELD', 'ELDN', 'ELF', 'ELLO', 'ELMD',
           'ELOX', 'ELP', 'ELS', 'ELSE', 'ELTK', 'ELVT', 'ELY', 'ELYS', 'EMAN', 'EMB', 'EMBD', 'EMCB', 'EMCF', 'EMD',
           'EMDV', 'EME', 'EMF', 'EMFM', 'EMGF', 'EMHY', 'EMIF', 'EMKR', 'EML', 'EMLC', 'EMLP', 'EMN', 'EMNT', 'EMO',
           'EMQQ', 'EMR', 'EMSG', 'EMTL', 'EMTY', 'EMX', 'EMXC', 'ENB', 'ENBL', 'ENDP', 'ENFAU', 'ENFR', 'ENG', 'ENIA',
           'ENIC', 'ENLC', 'ENLV', 'ENOB', 'ENOR', 'ENPH', 'ENR', 'ENS', 'ENSG', 'ENSV', 'ENTA', 'ENTG', 'ENTR', 'ENTX',
           'ENV', 'ENVA', 'ENVB', 'ENVIU', 'ENX', 'ENZ', 'ENZL', 'EOD', 'EOG', 'EOI', 'EOLS', 'EOS', 'EOSE', 'EOT',
           'EPAC', 'EPAM', 'EPAY', 'EPC', 'EPD', 'EPHE', 'EPI', 'EPIX', 'EPM', 'EPOL', 'EPP', 'EPR', 'EPRF', 'EPRT',
           'EPS', 'EPSN', 'EPU', 'EPV', 'EPZM', 'EQ', 'EQAL', 'EQBK', 'EQC', 'EQH', 'EQIX', 'EQL', 'EQNR', 'EQOS',
           'EQR', 'EQS', 'EQT', 'EQWL', 'EQX', 'ERC', 'ERESU', 'ERF', 'ERH', 'ERIC', 'ERIE', 'ERII', 'ERJ', 'ERM',
           'ERUS', 'ERX', 'ERY', 'ERYP', 'ES', 'ESBA', 'ESBK', 'ESCA', 'ESE', 'ESEA', 'ESG', 'ESGC', 'ESGD', 'ESGE',
           'ESGR', 'ESGU', 'ESGV', 'ESI', 'ESLT', 'ESML', 'ESNT', 'ESP', 'ESPO', 'ESPR', 'ESQ', 'ESRT', 'ESS', 'ESSA',
           'ESTA', 'ESTC', 'ESTE', 'ESXB', 'ET', 'ETAC', 'ETACU', 'ETB', 'ETG', 'ETH', 'ETHO', 'ETJ', 'ETM', 'ETN',
           'ETNB', 'ETO', 'ETON', 'ETR', 'ETRN', 'ETSY', 'ETTX', 'ETV', 'ETW', 'ETX', 'ETY', 'EUCRU', 'EUDG', 'EUFN',
           'EUM', 'EUO', 'EURL', 'EURN', 'EUSA', 'EUSC', 'EV', 'EVA', 'EVAX', 'EVBG', 'EVBN', 'EVC', 'EVER', 'EVF',
           'EVFM', 'EVG', 'EVGN', 'EVH', 'EVI', 'EVK', 'EVLO', 'EVM', 'EVN', 'EVOK', 'EVOL', 'EVOP', 'EVR', 'EVRG',
           'EVRI', 'EVT', 'EVTC', 'EVV', 'EVY', 'EW', 'EWA', 'EWBC', 'EWC', 'EWCO', 'EWD', 'EWG', 'EWGS', 'EWH', 'EWI',
           'EWJ', 'EWJE', 'EWJV', 'EWK', 'EWL', 'EWM', 'EWMC', 'EWN', 'EWO', 'EWP', 'EWQ', 'EWRE', 'EWS', 'EWSC', 'EWT',
           'EWU', 'EWUS', 'EWV', 'EWW', 'EWX', 'EWY', 'EWZ', 'EWZS', 'EXAS', 'EXC', 'EXD', 'EXEL', 'EXFO', 'EXG', 'EXI',
           'EXK', 'EXLS', 'EXN', 'EXP', 'EXPC', 'EXPD', 'EXPE', 'EXPI', 'EXPO', 'EXPR', 'EXR', 'EXTN', 'EXTR', 'EYE',
           'EYEG', 'EYEN', 'EYES', 'EYLD', 'EYPT', 'EZA', 'EZGO', 'EZJ', 'EZM', 'EZPW', 'EZU', 'F', 'FAAR', 'FAB',
           'FAD', 'FAF', 'FALN', 'FAM', 'FAMI', 'FAN', 'FANG', 'FANH', 'FARM', 'FARO', 'FAS', 'FAST', 'FAT', 'FATE',
           'FAUG', 'FAX', 'FAZ', 'FB', 'FBC', 'FBCG', 'FBCV', 'FBHS', 'FBIO', 'FBIZ', 'FBK', 'FBMS', 'FBNC', 'FBND',
           'FBP', 'FBRX', 'FBSS', 'FBT', 'FBZ', 'FC', 'FCA', 'FCACU', 'FCAL', 'FCAP', 'FCBC', 'FCBP', 'FCCO', 'FCCY',
           'FCEF', 'FCEL', 'FCF', 'FCFS', 'FCG', 'FCN', 'FCNCA', 'FCNCP', 'FCO', 'FCOM', 'FCOR', 'FCPT', 'FCRD', 'FCT',
           'FCTR', 'FCVT', 'FCX', 'FDD', 'FDEC', 'FDEM', 'FDEU', 'FDEV', 'FDG', 'FDHY', 'FDIS', 'FDIV', 'FDL', 'FDLO',
           'FDM', 'FDMO', 'FDMT', 'FDN', 'FDNI', 'FDP', 'FDRR', 'FDS', 'FDT', 'FDUS', 'FDVV', 'FDX', 'FE', 'FEDU',
           'FEI', 'FEIM', 'FELE', 'FEM', 'FEMB', 'FEMS', 'FEN', 'FENC', 'FENG', 'FENY', 'FEO', 'FEP', 'FET', 'FEUZ',
           'FEX', 'FEYE', 'FEZ', 'FF', 'FFA', 'FFBC', 'FFBW', 'FFC', 'FFEB', 'FFG', 'FFHG', 'FFHL', 'FFIC', 'FFIN',
           'FFIV', 'FFNW', 'FFR', 'FFSG', 'FFTG', 'FFTI', 'FFTY', 'FFWM', 'FGB', 'FGBI', 'FGD', 'FGEN', 'FGF', 'FGM',
           'FGNA', 'FGRO', 'FHB', 'FHI', 'FHLC', 'FHN', 'FHTX', 'FI', 'FIBK', 'FIBR', 'FICO', 'FID', 'FIDI', 'FIDU',
           'FIF', 'FIII', 'FIIIU', 'FILL', 'FINMU', 'FINV', 'FINX', 'FIS', 'FISI', 'FISR', 'FISV', 'FITB', 'FITE',
           'FIV', 'FIVA', 'FIVE', 'FIVG', 'FIVN', 'FIW', 'FIX', 'FIXD', 'FIXX', 'FIZZ', 'FJAN', 'FJP', 'FJUN', 'FKU',
           'FL', 'FLAU', 'FLAX', 'FLBR', 'FLC', 'FLCA', 'FLCB', 'FLCH', 'FLCO', 'FLDM', 'FLDR', 'FLEE', 'FLEX', 'FLGB',
           'FLGE', 'FLGR', 'FLGT', 'FLHK', 'FLHY', 'FLIA', 'FLIC', 'FLIN', 'FLIR', 'FLIY', 'FLJP', 'FLKR', 'FLL',
           'FLLV', 'FLMB', 'FLMN', 'FLMX', 'FLN', 'FLNG', 'FLNT', 'FLO', 'FLOT', 'FLOW', 'FLQH', 'FLQL', 'FLQM', 'FLR',
           'FLRN', 'FLRU', 'FLS', 'FLSW', 'FLT', 'FLTB', 'FLTR', 'FLUX', 'FLV', 'FLWS', 'FLXN', 'FLXS', 'FLY', 'FM',
           'FMAC', 'FMAG', 'FMAO', 'FMAT', 'FMAY', 'FMB', 'FMBH', 'FMBI', 'FMC', 'FMF', 'FMHI', 'FMIL', 'FMN', 'FMNB',
           'FMO', 'FMS', 'FMTX', 'FMX', 'FMY', 'FN', 'FNB', 'FNCB', 'FNCL', 'FND', 'FNDA', 'FNDB', 'FNDC', 'FNDE',
           'FNDF', 'FNDX', 'FNF', 'FNGD', 'FNGO', 'FNGS', 'FNGU', 'FNGZ', 'FNHC', 'FNI', 'FNK', 'FNKO', 'FNLC', 'FNOV',
           'FNV', 'FNWB', 'FNX', 'FNY', 'FOCS', 'FOE', 'FOF', 'FOLD', 'FONR', 'FOR', 'FORD', 'FORM', 'FORR', 'FORTY',
           'FOSL', 'FOUR', 'FOVL', 'FOX', 'FOXA', 'FOXF', 'FOXWU', 'FPAC', 'FPAY', 'FPE', 'FPEI', 'FPF', 'FPH', 'FPI',
           'FPL', 'FPRO', 'FPRX', 'FPX', 'FPXI', 'FQAL', 'FR', 'FRA', 'FRAK', 'FRBA', 'FRBK', 'FRC', 'FRD', 'FRDM',
           'FREE', 'FREL', 'FREQ', 'FRG', 'FRGI', 'FRHC', 'FRI', 'FRLG', 'FRLN', 'FRME', 'FRO', 'FROG', 'FRPH', 'FRPT',
           'FRSX', 'FRT', 'FRTA', 'FRX', 'FSBW', 'FSD', 'FSDC', 'FSFG', 'FSI', 'FSK', 'FSKR', 'FSLF', 'FSLR', 'FSLY',
           'FSM', 'FSMB', 'FSMD', 'FSP', 'FSR', 'FSRV', 'FSS', 'FSSIU', 'FST', 'FSTA', 'FSTR', 'FSTX', 'FSV', 'FSZ',
           'FT', 'FTA', 'FTAG', 'FTAI', 'FTC', 'FTCH', 'FTCS', 'FTCV', 'FTCVU', 'FTDR', 'FTEC', 'FTEK', 'FTF', 'FTFT',
           'FTGC', 'FTHI', 'FTHM', 'FTI', 'FTIV', 'FTIVU', 'FTK', 'FTLS', 'FTNT', 'FTOC', 'FTOCU', 'FTRI', 'FTS',
           'FTSD', 'FTSI', 'FTSL', 'FTSM', 'FTV', 'FTXD', 'FTXH', 'FTXL', 'FTXN', 'FTXO', 'FTXR', 'FUBO', 'FUL', 'FULC',
           'FULT', 'FUMB', 'FUN', 'FUNC', 'FUND', 'FUNL', 'FURY', 'FUSE', 'FUSN', 'FUTU', 'FUTY', 'FUV', 'FV', 'FVAL',
           'FVAM', 'FVC', 'FVCB', 'FVD', 'FVE', 'FVRR', 'FWDB', 'FWONA', 'FWONK', 'FWP', 'FWRD', 'FXA', 'FXB', 'FXC',
           'FXD', 'FXE', 'FXF', 'FXG', 'FXH', 'FXI', 'FXL', 'FXN', 'FXO', 'FXP', 'FXR', 'FXU', 'FXY', 'FXZ', 'FYC',
           'FYLD', 'FYT', 'FYX', 'G', 'GAA', 'GAB', 'GABC', 'GAIA', 'GAIN', 'GAL', 'GALT', 'GAM', 'GAMR', 'GAN', 'GASS',
           'GATO', 'GATX', 'GAU', 'GAZ', 'GB', 'GBAB', 'GBCI', 'GBDC', 'GBF', 'GBIL', 'GBIO', 'GBL', 'GBLI', 'GBNY',
           'GBR', 'GBS', 'GBT', 'GBUG', 'GBX', 'GCACU', 'GCBC', 'GCC', 'GCI', 'GCO', 'GCOR', 'GCOW', 'GCP', 'GCV', 'GD',
           'GDDY', 'GDEN', 'GDL', 'GDMA', 'GDO', 'GDOT', 'GDP', 'GDRX', 'GDS', 'GDV', 'GDX', 'GDXJ', 'GDXU', 'GDYN',
           'GE', 'GECC', 'GEF', 'GEL', 'GEM', 'GEN', 'GENC', 'GENE', 'GENY', 'GEO', 'GEOS', 'GER', 'GERM', 'GERN',
           'GES', 'GEVO', 'GF', 'GFED', 'GFF', 'GFI', 'GFL', 'GFN', 'GGAL', 'GGB', 'GGG', 'GGM', 'GGN', 'GGT', 'GGZ',
           'GH', 'GHACU', 'GHC', 'GHG', 'GHL', 'GHLD', 'GHM', 'GHSI', 'GHVI', 'GHY', 'GHYB', 'GHYG', 'GIB', 'GIFI',
           'GIGB', 'GIGE', 'GIGM', 'GII', 'GIII', 'GIK', 'GIL', 'GILD', 'GILT', 'GIM', 'GINN', 'GIS', 'GIX', 'GKOS',
           'GL', 'GLAD', 'GLBS', 'GLBZ', 'GLD', 'GLDD', 'GLDG', 'GLDI', 'GLDM', 'GLG', 'GLIN', 'GLL', 'GLMD', 'GLNG',
           'GLO', 'GLOB', 'GLOG', 'GLOP', 'GLP', 'GLPG', 'GLPI', 'GLQ', 'GLRE', 'GLSI', 'GLT', 'GLTO', 'GLTR', 'GLU',
           'GLUU', 'GLV', 'GLW', 'GLYC', 'GM', 'GMAB', 'GMBL', 'GMDA', 'GME', 'GMED', 'GMF', 'GMIIU', 'GMLP', 'GMOM',
           'GMRE', 'GMS', 'GNAF', 'GNCA', 'GNE', 'GNFT', 'GNK', 'GNL', 'GNLN', 'GNMA', 'GNMK', 'GNOG', 'GNOM', 'GNPK',
           'GNPX', 'GNR', 'GNRC', 'GNRS', 'GNSS', 'GNT', 'GNTX', 'GNTY', 'GNUS', 'GNW', 'GO', 'GOAC', 'GOAT', 'GOAU',
           'GOCO', 'GOED', 'GOEV', 'GOEX', 'GOF', 'GOGL', 'GOGO', 'GOL', 'GOLD', 'GOLF', 'GOOD', 'GOOG', 'GOOGL',
           'GOOS', 'GORO', 'GOSS', 'GOVT', 'GOVX', 'GP', 'GPC', 'GPI', 'GPK', 'GPL', 'GPM', 'GPMT', 'GPN', 'GPP',
           'GPRE', 'GPRK', 'GPRO', 'GPS', 'GPX', 'GQRE', 'GRA', 'GRAY', 'GRBK', 'GRC', 'GRCL', 'GREK', 'GRES', 'GRF',
           'GRFS', 'GRID', 'GRIL', 'GRIN', 'GRMN', 'GRNB', 'GRNQ', 'GROW', 'GRPN', 'GRSV', 'GRSVU', 'GRTS', 'GRTX',
           'GRU', 'GRUB', 'GRVY', 'GRWG', 'GRX', 'GS', 'GSAH', 'GSAQU', 'GSAT', 'GSBC', 'GSBD', 'GSEU', 'GSEW', 'GSG',
           'GSHD', 'GSIE', 'GSIT', 'GSJY', 'GSK', 'GSKY', 'GSL', 'GSLC', 'GSM', 'GSMG', 'GSP', 'GSS', 'GSSC', 'GSST',
           'GSUM', 'GSV', 'GSX', 'GSY', 'GT', 'GTE', 'GTEC', 'GTES', 'GTH', 'GTHX', 'GTIM', 'GTIP', 'GTLS', 'GTN',
           'GTO', 'GTS', 'GTT', 'GTY', 'GTYH', 'GUNR', 'GURE', 'GURU', 'GUSH', 'GUT', 'GVA', 'GVAL', 'GVI', 'GVIP',
           'GVP', 'GWAC', 'GWB', 'GWGH', 'GWPH', 'GWRE', 'GWRS', 'GWW', 'GWX', 'GXC', 'GXF', 'GXG', 'GXTG', 'GYC',
           'GYLD', 'GYRO', 'H', 'HA', 'HAAC', 'HAACU', 'HACK', 'HAE', 'HAFC', 'HAIL', 'HAIN', 'HAL', 'HALL', 'HALO',
           'HAP', 'HAPP', 'HARP', 'HAS', 'HASI', 'HAUZ', 'HAWX', 'HAYN', 'HBAN', 'HBB', 'HBCP', 'HBI', 'HBIO', 'HBM',
           'HBMD', 'HBNC', 'HBP', 'HBT', 'HCA', 'HCAP', 'HCAQ', 'HCAR', 'HCARU', 'HCAT', 'HCC', 'HCCCU', 'HCCI', 'HCDI',
           'HCHC', 'HCI', 'HCICU', 'HCIIU', 'HCKT', 'HCM', 'HCSG', 'HD', 'HDAW', 'HDB', 'HDEF', 'HDG', 'HDGE', 'HDLB',
           'HDMV', 'HDSN', 'HDV', 'HE', 'HEAR', 'HEC', 'HECCU', 'HEDJ', 'HEEM', 'HEES', 'HEFA', 'HEGD', 'HEI', 'HEI-A',
           'HELE', 'HELX', 'HEP', 'HEPA', 'HEQ', 'HERO', 'HES', 'HESM', 'HEWG', 'HEWJ', 'HEWU', 'HEXO', 'HEZU', 'HFBL',
           'HFC', 'HFEN', 'HFFG', 'HFRO', 'HFWA', 'HFXI', 'HGBL', 'HGEN', 'HGLB', 'HGSH', 'HGV', 'HHC', 'HHR', 'HI',
           'HIBB', 'HIBL', 'HIBS', 'HIE', 'HIFS', 'HIG', 'HIGA', 'HIHO', 'HII', 'HIL', 'HIMS', 'HIMX', 'HIO', 'HIPS',
           'HIW', 'HIX', 'HJLI', 'HKIB', 'HL', 'HLAHU', 'HLAL', 'HLF', 'HLG', 'HLI', 'HLIO', 'HLIT', 'HLNE', 'HLT',
           'HLX', 'HLXA', 'HMC', 'HMCOU', 'HMHC', 'HMI', 'HMLP', 'HMN', 'HMNF', 'HMOP', 'HMPT', 'HMST', 'HMSY', 'HMTV',
           'HMY', 'HNDL', 'HNGR', 'HNI', 'HNNA', 'HNP', 'HNRG', 'HNW', 'HOFT', 'HOFV', 'HOG', 'HOL', 'HOLD', 'HOLI',
           'HOLUU', 'HOLX', 'HOMB', 'HOME', 'HOMZ', 'HON', 'HONE', 'HOOK', 'HOPE', 'HOTH', 'HOV', 'HOVNP', 'HP', 'HPE',
           'HPF', 'HPI', 'HPP', 'HPQ', 'HPR', 'HPS', 'HQH', 'HQI', 'HQL', 'HQY', 'HR', 'HRB', 'HRC', 'HRI', 'HRL',
           'HRMY', 'HROW', 'HRTG', 'HRTX', 'HRZN', 'HSAQ', 'HSBC', 'HSC', 'HSCZ', 'HSDT', 'HSIC', 'HSII', 'HSKA',
           'HSON', 'HSRT', 'HST', 'HSTM', 'HSTO', 'HSY', 'HT', 'HTA', 'HTAB', 'HTBI', 'HTBK', 'HTBX', 'HTD', 'HTEC',
           'HTGC', 'HTGM', 'HTH', 'HTHT', 'HTLD', 'HTLF', 'HTOO', 'HTRB', 'HTY', 'HUBB', 'HUBG', 'HUBS', 'HUDI', 'HUGE',
           'HUIZ', 'HUM', 'HUN', 'HURC', 'HURN', 'HUSA', 'HUSN', 'HUSV', 'HUYA', 'HVT', 'HWBK', 'HWC', 'HWCC', 'HWKN',
           'HWM', 'HX', 'HXL', 'HY', 'HYB', 'HYD', 'HYDB', 'HYDW', 'HYEM', 'HYFM', 'HYG', 'HYGH', 'HYGV', 'HYHG', 'HYI',
           'HYLB', 'HYLD', 'HYLN', 'HYLS', 'HYLV', 'HYMB', 'HYMC', 'HYRE', 'HYS', 'HYT', 'HYXU', 'HYZD', 'HZAC', 'HZN',
           'HZNP', 'HZO', 'HZON', 'IAA', 'IAC', 'IAE', 'IAF', 'IAG', 'IAGG', 'IAI', 'IAK', 'IART', 'IAT', 'IAU', 'IAUF',
           'IBA', 'IBB', 'IBBJ', 'IBCP', 'IBD', 'IBDD', 'IBDM', 'IBDN', 'IBDO', 'IBDP', 'IBDQ', 'IBDR', 'IBDS', 'IBDT',
           'IBDU', 'IBEX', 'IBHA', 'IBHB', 'IBHC', 'IBHD', 'IBHE', 'IBIO', 'IBKR', 'IBM', 'IBMJ', 'IBMK', 'IBML',
           'IBMM', 'IBMN', 'IBMO', 'IBMP', 'IBMQ', 'IBN', 'IBND', 'IBOC', 'IBP', 'IBTA', 'IBTB', 'IBTD', 'IBTE', 'IBTG',
           'IBTJ', 'IBTX', 'IBUY', 'ICAD', 'ICBK', 'ICCC', 'ICD', 'ICE', 'ICF', 'ICFI', 'ICHR', 'ICL', 'ICLK', 'ICLN',
           'ICLR', 'ICMB', 'ICOL', 'ICON', 'ICOW', 'ICPT', 'ICSH', 'ICUI', 'ICVT', 'ID', 'IDA', 'IDCC', 'IDE', 'IDEV',
           'IDEX', 'IDHD', 'IDHQ', 'IDIV', 'IDLV', 'IDMO', 'IDN', 'IDNA', 'IDOG', 'IDRA', 'IDRV', 'IDT', 'IDU', 'IDV',
           'IDX', 'IDXG', 'IDXX', 'IDYA', 'IEA', 'IEC', 'IECS', 'IEDI', 'IEF', 'IEFA', 'IEHS', 'IEI', 'IEIH', 'IEME',
           'IEMG', 'IEO', 'IEP', 'IESC', 'IETC', 'IEUR', 'IEUS', 'IEV', 'IEX', 'IEZ', 'IFF', 'IFGL', 'IFMK', 'IFN',
           'IFRA', 'IFRX', 'IFS', 'IFV', 'IG', 'IGA', 'IGAC', 'IGBH', 'IGC', 'IGD', 'IGE', 'IGEB', 'IGF', 'IGHG', 'IGI',
           'IGIB', 'IGLB', 'IGM', 'IGMS', 'IGN', 'IGNYU', 'IGOV', 'IGR', 'IGRO', 'IGSB', 'IGT', 'IGV', 'IH', 'IHAK',
           'IHC', 'IHD', 'IHDG', 'IHE', 'IHF', 'IHG', 'IHI', 'IHIT', 'IHRT', 'IHT', 'IHY', 'IID', 'IIF', 'IIGD', 'IIGV',
           'III', 'IIIIU', 'IIIN', 'IIIV', 'IIM', 'IIN', 'IIPR', 'IIVI', 'IJAN', 'IJH', 'IJJ', 'IJK', 'IJR', 'IJS',
           'IJT', 'IJUL', 'IKNX', 'IKT', 'ILF', 'ILMN', 'ILPT', 'ILTB', 'IMAB', 'IMAC', 'IMAX', 'IMBI', 'IMGN', 'IMH',
           'IMKTA', 'IMLP', 'IMMP', 'IMMR', 'IMNM', 'IMO', 'IMOM', 'IMOS', 'IMRA', 'IMRN', 'IMTB', 'IMTE', 'IMTM',
           'IMTX', 'IMUX', 'IMV', 'IMVT', 'IMXI', 'INAQ', 'INBK', 'INBX', 'INCO', 'INCY', 'INDA', 'INDB', 'INDL',
           'INDO', 'INDS', 'INDY', 'INFI', 'INFL', 'INFN', 'INFO', 'INFR', 'INFU', 'INFY', 'ING', 'INGN', 'INGR',
           'INKM', 'INM', 'INMB', 'INMD', 'INN', 'INO', 'INOD', 'INOV', 'INPX', 'INS', 'INSE', 'INSG', 'INSI', 'INSM',
           'INSP', 'INSW', 'INT', 'INTC', 'INTF', 'INTG', 'INTT', 'INTU', 'INTZ', 'INUV', 'INVA', 'INVE', 'INVH',
           'INZY', 'IO', 'IONS', 'IOO', 'IOR', 'IOSP', 'IOVA', 'IP', 'IPA', 'IPAC', 'IPAR', 'IPAY', 'IPDN', 'IPFF',
           'IPG', 'IPGP', 'IPHA', 'IPHI', 'IPI', 'IPKW', 'IPLDP', 'IPO', 'IPOD', 'IPOE', 'IPOF', 'IPOS', 'IPV', 'IPWR',
           'IQ', 'IQDE', 'IQDF', 'IQDG', 'IQDY', 'IQI', 'IQIN', 'IQLT', 'IQSI', 'IQV', 'IR', 'IRBO', 'IRBT', 'IRCP',
           'IRDM', 'IRIX', 'IRL', 'IRM', 'IRMD', 'IROQ', 'IRR', 'IRS', 'IRT', 'IRTC', 'IRWD', 'ISBC', 'ISCF', 'ISD',
           'ISDR', 'ISDX', 'ISEE', 'ISHG', 'ISIG', 'ISMD', 'ISNS', 'ISR', 'ISRA', 'ISRG', 'ISSC', 'ISTB', 'ISTR',
           'ISUN', 'IT', 'ITA', 'ITACU', 'ITB', 'ITCB', 'ITCI', 'ITEQ', 'ITGR', 'ITHXU', 'ITI', 'ITIC', 'ITM', 'ITMR',
           'ITOS', 'ITOT', 'ITP', 'ITQRU', 'ITRG', 'ITRI', 'ITRM', 'ITRN', 'ITT', 'ITUB', 'ITW', 'IUS', 'IUSB', 'IUSG',
           'IUSS', 'IUSV', 'IVA', 'IVAC', 'IVAL', 'IVC', 'IVE', 'IVES', 'IVH', 'IVLU', 'IVOG', 'IVOL', 'IVOO', 'IVOV',
           'IVR', 'IVV', 'IVW', 'IVZ', 'IWB', 'IWC', 'IWD', 'IWF', 'IWL', 'IWM', 'IWN', 'IWO', 'IWP', 'IWR', 'IWS',
           'IWV', 'IWX', 'IWY', 'IX', 'IXC', 'IXG', 'IXJ', 'IXN', 'IXP', 'IXUS', 'IYC', 'IYE', 'IYF', 'IYG', 'IYH',
           'IYJ', 'IYK', 'IYLD', 'IYM', 'IYR', 'IYT', 'IYW', 'IYY', 'IYZ', 'IZEA', 'IZRL', 'J', 'JACK', 'JAGG', 'JAGX',
           'JAKK', 'JAMF', 'JAN', 'JANZ', 'JAX', 'JAZZ', 'JBGS', 'JBHT', 'JBL', 'JBLU', 'JBSS', 'JBT', 'JCE', 'JCI',
           'JCICU', 'JCO', 'JCOM', 'JCPB', 'JCS', 'JCTCF', 'JD', 'JDD', 'JDIV', 'JDST', 'JE', 'JEF', 'JELD', 'JEPI',
           'JEQ', 'JETS', 'JFIN', 'JFR', 'JFU', 'JG', 'JGH', 'JHEM', 'JHG', 'JHI', 'JHMC', 'JHMD', 'JHME', 'JHMF',
           'JHMH', 'JHMI', 'JHML', 'JHMM', 'JHMT', 'JHS', 'JHSC', 'JHX', 'JIG', 'JIGB', 'JILL', 'JJC', 'JJG', 'JJN',
           'JJSF', 'JKD', 'JKE', 'JKF', 'JKG', 'JKH', 'JKHY', 'JKI', 'JKJ', 'JKK', 'JKL', 'JKS', 'JLL', 'JLS', 'JMBS',
           'JMIA', 'JMIN', 'JMM', 'JMOM', 'JMP', 'JMST', 'JMUB', 'JNCE', 'JNJ', 'JNK', 'JNPR', 'JNUG', 'JO', 'JOB',
           'JOBS', 'JOE', 'JOET', 'JOF', 'JOFFU', 'JOUT', 'JOYY', 'JP', 'JPC', 'JPEM', 'JPHY', 'JPI', 'JPIN', 'JPM',
           'JPMB', 'JPME', 'JPN', 'JPS', 'JPSE', 'JPST', 'JPUS', 'JPXN', 'JQC', 'JQUA', 'JRI', 'JRJC', 'JRO', 'JRS',
           'JRSH', 'JRVR', 'JSD', 'JSMD', 'JSML', 'JSTC', 'JT', 'JTA', 'JTD', 'JUPW', 'JUST', 'JVA', 'JVAL', 'JW-A',
           'JWN', 'JWS', 'JXI', 'JYAC', 'JYNT', 'K', 'KAI', 'KAIRU', 'KALA', 'KALL', 'KALU', 'KALV', 'KAMN', 'KAPR',
           'KAR', 'KARS', 'KB', 'KBA', 'KBAL', 'KBE', 'KBH', 'KBNT', 'KBR', 'KBSF', 'KBWB', 'KBWD', 'KBWP', 'KBWR',
           'KBWY', 'KC', 'KCE', 'KDFI', 'KDMN', 'KDNY', 'KDP', 'KE', 'KELYA', 'KEMQ', 'KEN', 'KEP', 'KEQU', 'KERN',
           'KESG', 'KEX', 'KEY', 'KEYS', 'KF', 'KFFB', 'KFRC', 'KFS', 'KFY', 'KFYP', 'KGC', 'KGRN', 'KHC', 'KIDS',
           'KIE', 'KIM', 'KIN', 'KINS', 'KINZ', 'KINZU', 'KIO', 'KIQ', 'KIRK', 'KJAN', 'KJUL', 'KKR', 'KL', 'KLAC',
           'KLAQU', 'KLCD', 'KLDO', 'KLDW', 'KLIC', 'KLR', 'KLXE', 'KMB', 'KMDA', 'KMF', 'KMI', 'KMPH', 'KMPR', 'KMT',
           'KMX', 'KN', 'KNDI', 'KNG', 'KNGS', 'KNL', 'KNOP', 'KNSA', 'KNSL', 'KNTE', 'KNX', 'KO', 'KOCT', 'KOD',
           'KODK', 'KOF', 'KOIN', 'KOKU', 'KOLD', 'KOMP', 'KOP', 'KOPN', 'KOR', 'KORP', 'KORU', 'KOS', 'KOSS', 'KPTI',
           'KR', 'KRA', 'KRBN', 'KRC', 'KRE', 'KREF', 'KRG', 'KRKR', 'KRMA', 'KRMD', 'KRNLU', 'KRNT', 'KRNY', 'KRO',
           'KRON', 'KROS', 'KRP', 'KRTX', 'KRUS', 'KRYS', 'KSA', 'KSCD', 'KSM', 'KSMTU', 'KSPN', 'KSS', 'KSTR', 'KSU',
           'KT', 'KTB', 'KTCC', 'KTF', 'KTOS', 'KTRA', 'KUKE', 'KURA', 'KURE', 'KVHI', 'KW', 'KWEB', 'KWR', 'KXI',
           'KXIN', 'KYMR', 'KYN', 'KZIA', 'KZR', 'L', 'LABD', 'LABP', 'LABU', 'LAC', 'LACQ', 'LAD', 'LADR', 'LAIX',
           'LAKE', 'LAMR', 'LANC', 'LAND', 'LANDM', 'LANDO', 'LARK', 'LASR', 'LATN', 'LAUR', 'LAWS', 'LAZ', 'LAZR',
           'LAZY', 'LB', 'LBAI', 'LBC', 'LBJ', 'LBRDA', 'LBRDK', 'LBRT', 'LBTYA', 'LBTYK', 'LC', 'LCG', 'LCI', 'LCII',
           'LCNB', 'LCTX', 'LCUT', 'LCY', 'LCYAU', 'LDEM', 'LDL', 'LDOS', 'LDP', 'LDRS', 'LDSF', 'LDUR', 'LE', 'LEA',
           'LEAD', 'LEAF', 'LEAP', 'LECO', 'LEDS', 'LEE', 'LEG', 'LEGH', 'LEGN', 'LEGOU', 'LEGR', 'LEJU', 'LEMB', 'LEN',
           'LEN-B', 'LEO', 'LESL', 'LEU', 'LEVI', 'LEVL', 'LEXX', 'LFC', 'LFEQ', 'LFT', 'LFTR', 'LFTRU', 'LFUS', 'LFVN',
           'LGF-A', 'LGF-B', 'LGH', 'LGHL', 'LGI', 'LGIH', 'LGL', 'LGLV', 'LGND', 'LGOV', 'LGVW', 'LH', 'LHCG', 'LHDX',
           'LHX', 'LI', 'LIFE', 'LII', 'LILA', 'LILAK', 'LIN', 'LINC', 'LIND', 'LINX', 'LIQT', 'LIT', 'LITB', 'LITE',
           'LIVE', 'LIVN', 'LIVX', 'LIZI', 'LJAQU', 'LJPC', 'LKCO', 'LKFN', 'LKOR', 'LKQ', 'LL', 'LLIT', 'LLNW', 'LLY',
           'LMACU', 'LMAT', 'LMB', 'LMBS', 'LMFA', 'LMND', 'LMNL', 'LMNR', 'LMNX', 'LMPX', 'LMRK', 'LMT', 'LNC', 'LND',
           'LNDC', 'LNFA', 'LNG', 'LNGR', 'LNN', 'LNSR', 'LNT', 'LNTH', 'LOAC', 'LOAN', 'LOB', 'LOCO', 'LODE', 'LOGC',
           'LOGI', 'LOKB', 'LOMA', 'LOOP', 'LOPE', 'LORL', 'LOTZ', 'LOUP', 'LOV', 'LOVE', 'LOW', 'LOWC', 'LPCN', 'LPG',
           'LPI', 'LPL', 'LPLA', 'LPRO', 'LPSN', 'LPTH', 'LPTX', 'LPX', 'LQD', 'LQDA', 'LQDH', 'LQDI', 'LQDT', 'LRCX',
           'LRGE', 'LRGF', 'LRMR', 'LRN', 'LRNZ', 'LSAF', 'LSAQ', 'LSBK', 'LSCC', 'LSEA', 'LSF', 'LSI', 'LSPD', 'LSST',
           'LSTR', 'LSXMA', 'LSXMK', 'LTBR', 'LTC', 'LTHM', 'LTPZ', 'LTRN', 'LTRPA', 'LTRPB', 'LTRX', 'LU', 'LUB',
           'LULU', 'LUMN', 'LUMO', 'LUNA', 'LUNG', 'LUV', 'LUXAU', 'LVHD', 'LVHI', 'LVS', 'LW', 'LWACU', 'LWAY', 'LX',
           'LXEH', 'LXFR', 'LXP', 'LXRX', 'LXU', 'LYB', 'LYFT', 'LYG', 'LYL', 'LYRA', 'LYTS', 'LYV', 'LZB', 'M', 'MA',
           'MAA', 'MAAC', 'MAACU', 'MAC', 'MACK', 'MACUU', 'MAG', 'MAGA', 'MAGS', 'MAIN', 'MAN', 'MANH', 'MANT', 'MANU',
           'MAR', 'MARA', 'MARK', 'MARPS', 'MAS', 'MASI', 'MASS', 'MAT', 'MATW', 'MATX', 'MAV', 'MAX', 'MAXN', 'MAXR',
           'MAYS', 'MBB', 'MBI', 'MBII', 'MBIN', 'MBIO', 'MBNKP', 'MBOT', 'MBRX', 'MBSD', 'MBT', 'MBUU', 'MBWM', 'MC',
           'MCA', 'MCAC', 'MCADU', 'MCB', 'MCBC', 'MCBS', 'MCD', 'MCEF', 'MCF', 'MCFE', 'MCFT', 'MCHI', 'MCHP', 'MCHX',
           'MCI', 'MCK', 'MCN', 'MCO', 'MCR', 'MCRB', 'MCRI', 'MCS', 'MCY', 'MD', 'MDB', 'MDC', 'MDCA', 'MDGL', 'MDGS',
           'MDIA', 'MDIV', 'MDJH', 'MDLA', 'MDLY', 'MDLZ', 'MDNA', 'MDP', 'MDRR', 'MDRX', 'MDT', 'MDU', 'MDVL', 'MDWD',
           'MDXG', 'MDY', 'MDYG', 'MDYV', 'MEAR', 'MEC', 'MED', 'MEDP', 'MEDS', 'MEG', 'MEI', 'MEIP', 'MELI', 'MEN',
           'MEOH', 'MERC', 'MESA', 'MESO', 'MET', 'METC', 'METX', 'MEXX', 'MFA', 'MFC', 'MFD', 'MFDX', 'MFEM', 'MFG',
           'MFGP', 'MFH', 'MFIN', 'MFL', 'MFM', 'MFMS', 'MFNC', 'MFT', 'MFUS', 'MFV', 'MG', 'MGA', 'MGC', 'MGEE', 'MGF',
           'MGI', 'MGIC', 'MGK', 'MGLN', 'MGM', 'MGNI', 'MGNX', 'MGP', 'MGPI', 'MGRC', 'MGTA', 'MGTX', 'MGU', 'MGV',
           'MGY', 'MGYR', 'MHD', 'MHE', 'MHF', 'MHH', 'MHI', 'MHK', 'MHLD', 'MHN', 'MHO', 'MIC', 'MICT', 'MIDD', 'MIDU',
           'MIE', 'MIK', 'MILN', 'MIME', 'MIN', 'MINC', 'MIND', 'MINT', 'MIRM', 'MIST', 'MITK', 'MITO', 'MITT', 'MIXT',
           'MIY', 'MJ', 'MJO', 'MKC', 'MKD', 'MKGI', 'MKL', 'MKSI', 'MKTX', 'MLAB', 'MLAC', 'MLCO', 'MLHR', 'MLI',
           'MLM', 'MLN', 'MLND', 'MLP', 'MLPA', 'MLPB', 'MLPX', 'MLR', 'MLSS', 'MLVF', 'MMAC', 'MMC', 'MMD', 'MMI',
           'MMIN', 'MMIT', 'MMLP', 'MMM', 'MMP', 'MMS', 'MMSI', 'MMT', 'MMTM', 'MMU', 'MMX', 'MMYT', 'MN', 'MNA',
           'MNDO', 'MNKD', 'MNOV', 'MNP', 'MNPR', 'MNR', 'MNRL', 'MNRO', 'MNSB', 'MNSO', 'MNST', 'MNTK', 'MNTX', 'MO',
           'MOAT', 'MOD', 'MODN', 'MODV', 'MOFG', 'MOG-A', 'MOGO', 'MOGU', 'MOH', 'MOHO', 'MOM', 'MOMO', 'MOO', 'MOON',
           'MOR', 'MORF', 'MORN', 'MORT', 'MOS', 'MOSY', 'MOTI', 'MOTNU', 'MOTO', 'MOTS', 'MOTV', 'MOV', 'MOXC', 'MP',
           'MPA', 'MPAA', 'MPB', 'MPC', 'MPLN', 'MPLX', 'MPV', 'MPW', 'MPWR', 'MPX', 'MQT', 'MQY', 'MRACU', 'MRAD',
           'MRAM', 'MRC', 'MRCC', 'MRCY', 'MREO', 'MRIN', 'MRK', 'MRKR', 'MRLN', 'MRM', 'MRNA', 'MRNS', 'MRO', 'MRSN',
           'MRTN', 'MRTX', 'MRUS', 'MRVI', 'MRVL', 'MS', 'MSA', 'MSB', 'MSBI', 'MSC', 'MSCI', 'MSD', 'MSEX', 'MSFT',
           'MSGE', 'MSGM', 'MSGN', 'MSGS', 'MSI', 'MSM', 'MSN', 'MSON', 'MSOS', 'MSP', 'MSTB', 'MSTR', 'MSVB', 'MT',
           'MTA', 'MTACU', 'MTB', 'MTBC', 'MTC', 'MTCH', 'MTCR', 'MTD', 'MTDR', 'MTEM', 'MTEX', 'MTG', 'MTGP', 'MTH',
           'MTL', 'MTLS', 'MTN', 'MTNB', 'MTOR', 'MTP', 'MTR', 'MTRN', 'MTRX', 'MTSC', 'MTSI', 'MTSL', 'MTT', 'MTUM',
           'MTW', 'MTX', 'MTZ', 'MU', 'MUA', 'MUB', 'MUC', 'MUDS', 'MUE', 'MUFG', 'MUH', 'MUI', 'MUJ', 'MUNI', 'MUR',
           'MUS', 'MUSA', 'MUST', 'MUX', 'MVBF', 'MVF', 'MVIS', 'MVO', 'MVT', 'MVV', 'MWA', 'MWK', 'MX', 'MXC', 'MXDU',
           'MXE', 'MXF', 'MXI', 'MXIM', 'MXL', 'MYC', 'MYD', 'MYE', 'MYF', 'MYFW', 'MYGN', 'MYI', 'MYJ', 'MYN', 'MYO',
           'MYOV', 'MYRG', 'MYSZ', 'MYT', 'MYTE', 'MYY', 'MZA', 'MZZ', 'NAACU', 'NAC', 'NACP', 'NAD', 'NAII', 'NAIL',
           'NAK', 'NAKD', 'NAN', 'NANR', 'NAOV', 'NAPR', 'NARI', 'NAT', 'NATH', 'NATI', 'NATR', 'NAV', 'NAVB', 'NAVI',
           'NAZ', 'NBAC', 'NBB', 'NBEV', 'NBH', 'NBHC', 'NBIX', 'NBLX', 'NBN', 'NBO', 'NBR', 'NBRV', 'NBSE', 'NBTB',
           'NBTX', 'NBW', 'NBY', 'NC', 'NCA', 'NCB', 'NCBS', 'NCLH', 'NCMI', 'NCNA', 'NCNO', 'NCR', 'NCSM', 'NCTY',
           'NCV', 'NCZ', 'NDAQ', 'NDLS', 'NDP', 'NDRA', 'NDSN', 'NEA', 'NEAR', 'NEBCU', 'NEE', 'NEM', 'NEN', 'NEO',
           'NEOG', 'NEON', 'NEOS', 'NEP', 'NEPH', 'NEPT', 'NERD', 'NERV', 'NES', 'NESR', 'NET', 'NETE', 'NETL', 'NEU',
           'NEV', 'NEW', 'NEWA', 'NEWR', 'NEWT', 'NEX', 'NEXA', 'NEXT', 'NFBK', 'NFE', 'NFG', 'NFH', 'NFJ', 'NFLT',
           'NFLX', 'NFRA', 'NG', 'NGA', 'NGACU', 'NGD', 'NGE', 'NGG', 'NGL', 'NGM', 'NGMS', 'NGS', 'NGVC', 'NGVT', 'NH',
           'NHA', 'NHC', 'NHF', 'NHI', 'NHIC', 'NHICU', 'NHLD', 'NHS', 'NHTC', 'NI', 'NIB', 'NICE', 'NICK', 'NID',
           'NIE', 'NIM', 'NINE', 'NIO', 'NIQ', 'NISN', 'NIU', 'NJAN', 'NJR', 'NJUL', 'NJV', 'NK', 'NKE', 'NKG', 'NKLA',
           'NKSH', 'NKTR', 'NKTX', 'NKX', 'NL', 'NLOK', 'NLR', 'NLS', 'NLSN', 'NLSP', 'NLTX', 'NLY', 'NM', 'NMCI',
           'NMCO', 'NMFC', 'NMI', 'NMIH', 'NML', 'NMM', 'NMR', 'NMRD', 'NMRK', 'NMS', 'NMT', 'NMTR', 'NMY', 'NMZ',
           'NNA', 'NNBR', 'NNDM', 'NNI', 'NNN', 'NNOX', 'NNVC', 'NNY', 'NOA', 'NOACU', 'NOAH', 'NOBL', 'NOC', 'NOCT',
           'NODK', 'NOG', 'NOK', 'NOM', 'NOMD', 'NORW', 'NOV', 'NOVA', 'NOVN', 'NOVT', 'NOW', 'NP', 'NPA', 'NPAUU',
           'NPK', 'NPN', 'NPO', 'NPTN', 'NPV', 'NQP', 'NR', 'NRBO', 'NRC', 'NREF', 'NRG', 'NRGD', 'NRGU', 'NRGX',
           'NRIM', 'NRIX', 'NRK', 'NRO', 'NRP', 'NRT', 'NRZ', 'NS', 'NSA', 'NSC', 'NSCO', 'NSEC', 'NSIT', 'NSL', 'NSP',
           'NSPR', 'NSSC', 'NSTG', 'NSYS', 'NTAP', 'NTB', 'NTCO', 'NTCT', 'NTEC', 'NTES', 'NTG', 'NTGR', 'NTIC', 'NTIP',
           'NTLA', 'NTN', 'NTNX', 'NTP', 'NTR', 'NTRA', 'NTRS', 'NTST', 'NTSX', 'NTUS', 'NTWK', 'NTZ', 'NUAG', 'NUAN',
           'NUBD', 'NUDM', 'NUE', 'NUEM', 'NUGT', 'NUHY', 'NULC', 'NULG', 'NULV', 'NUM', 'NUMG', 'NUMV', 'NUO', 'NURE',
           'NURO', 'NUS', 'NUSA', 'NUSC', 'NUSI', 'NUV', 'NUVA', 'NUW', 'NUZE', 'NVAX', 'NVCN', 'NVCR', 'NVDA', 'NVEC',
           'NVEE', 'NVFY', 'NVG', 'NVGS', 'NVIV', 'NVMI', 'NVO', 'NVR', 'NVRO', 'NVS', 'NVST', 'NVT', 'NVTA', 'NWBI',
           'NWE', 'NWFL', 'NWG', 'NWHM', 'NWL', 'NWLI', 'NWN', 'NWPX', 'NWS', 'NWSA', 'NX', 'NXC', 'NXE', 'NXGN', 'NXJ',
           'NXN', 'NXP', 'NXPI', 'NXQ', 'NXR', 'NXRT', 'NXST', 'NXTC', 'NXTD', 'NXTG', 'NYCB', 'NYF', 'NYMT', 'NYMX',
           'NYT', 'NYV', 'NZF', 'O', 'OAS', 'OBAS', 'OBCI', 'OBLG', 'OBLN', 'OBNK', 'OBSV', 'OC', 'OCA', 'OCAXU', 'OCC',
           'OCCI', 'OCDX', 'OCFC', 'OCFT', 'OCG', 'OCGN', 'OCN', 'OCSI', 'OCSL', 'OCUL', 'OCUP', 'OCX', 'ODC', 'ODFL',
           'ODP', 'ODT', 'OEC', 'OEF', 'OEG', 'OESX', 'OEUR', 'OFC', 'OFED', 'OFG', 'OFIX', 'OFLX', 'OFS', 'OGE',
           'OGEN', 'OGI', 'OGIG', 'OGS', 'OHI', 'OI', 'OIA', 'OIBR-C', 'OIH', 'OII', 'OIIM', 'OILK', 'OIS', 'OKE',
           'OKTA', 'OLB', 'OLD', 'OLED', 'OLLI', 'OLMA', 'OLN', 'OLP', 'OM', 'OMAB', 'OMC', 'OMCL', 'OMEG', 'OMER',
           'OMEX', 'OMF', 'OMFL', 'OMFS', 'OMI', 'OMP', 'ON', 'ONB', 'ONCR', 'ONCS', 'ONCT', 'ONCY', 'ONDS', 'ONE',
           'ONEM', 'ONEO', 'ONEQ', 'ONEV', 'ONEW', 'ONEY', 'ONLN', 'ONTF', 'ONTO', 'ONTX', 'ONVO', 'OOMA', 'OPBK',
           'OPCH', 'OPEN', 'OPER', 'OPGN', 'OPHC', 'OPI', 'OPK', 'OPNT', 'OPOF', 'OPP', 'OPRA', 'OPRT', 'OPRX', 'OPT',
           'OPTN', 'OPTT', 'OPY', 'OR', 'ORA', 'ORAN', 'ORBC', 'ORC', 'ORCC', 'ORCL', 'ORGO', 'ORGS', 'ORI', 'ORIC',
           'ORLA', 'ORLY', 'ORMP', 'ORN', 'ORPH', 'ORRF', 'ORTX', 'OSBC', 'OSCV', 'OSG', 'OSH', 'OSIS', 'OSK', 'OSMT',
           'OSN', 'OSPN', 'OSS', 'OSTK', 'OSTRU', 'OSUR', 'OSW', 'OTEL', 'OTEX', 'OTIC', 'OTIS', 'OTLK', 'OTRK', 'OTTR',
           'OUNZ', 'OUSA', 'OUSM', 'OUT', 'OVB', 'OVBC', 'OVID', 'OVL', 'OVLH', 'OVLY', 'OVM', 'OVT', 'OVV', 'OXBR',
           'OXFD', 'OXLC', 'OXM', 'OXSQ', 'OXY', 'OYST', 'OZK', 'OZON', 'PAA', 'PAAS', 'PAC', 'PACB', 'PACK', 'PACW',
           'PAE', 'PAG', 'PAGP', 'PAGS', 'PAHC', 'PAI', 'PAICU', 'PAK', 'PALL', 'PAM', 'PAND', 'PANL', 'PANW', 'PAPR',
           'PAR', 'PARR', 'PASG', 'PATI', 'PATK', 'PAUG', 'PAVE', 'PAVM', 'PAWZ', 'PAX', 'PAYA', 'PAYC', 'PAYS', 'PAYX',
           'PB', 'PBA', 'PBCT', 'PBD', 'PBDM', 'PBE', 'PBF', 'PBFS', 'PBFX', 'PBH', 'PBHC', 'PBI', 'PBIP', 'PBJ',
           'PBLA', 'PBND', 'PBP', 'PBPB', 'PBR', 'PBR-A', 'PBS', 'PBSM', 'PBT', 'PBTP', 'PBTS', 'PBUS', 'PBW', 'PBYI',
           'PCAR', 'PCB', 'PCEF', 'PCF', 'PCG', 'PCGU', 'PCH', 'PCI', 'PCK', 'PCM', 'PCN', 'PCOM', 'PCPL', 'PCQ',
           'PCRX', 'PCSA', 'PCSB', 'PCTI', 'PCTY', 'PCVX', 'PCY', 'PCYG', 'PCYO', 'PD', 'PDAC', 'PDBC', 'PDCE', 'PDCO',
           'PDD', 'PDEC', 'PDEX', 'PDFS', 'PDI', 'PDLB', 'PDM', 'PDN', 'PDO', 'PDP', 'PDS', 'PDSB', 'PDT', 'PEAK',
           'PEB', 'PEBK', 'PEBO', 'PED', 'PEG', 'PEGA', 'PEI', 'PEJ', 'PEN', 'PENN', 'PEO', 'PEP', 'PERI', 'PESI',
           'PETQ', 'PETS', 'PETZ', 'PEX', 'PEY', 'PEZ', 'PFBC', 'PFBI', 'PFC', 'PFD', 'PFE', 'PFEB', 'PFF', 'PFFA',
           'PFFD', 'PFFL', 'PFFR', 'PFG', 'PFGC', 'PFHD', 'PFI', 'PFIE', 'PFIG', 'PFIN', 'PFIS', 'PFL', 'PFLD', 'PFLT',
           'PFM', 'PFMT', 'PFN', 'PFO', 'PFPT', 'PFS', 'PFSI', 'PFSW', 'PFXF', 'PG', 'PGAL', 'PGC', 'PGEN', 'PGF',
           'PGHY', 'PGJ', 'PGNY', 'PGP', 'PGR', 'PGRE', 'PGTI', 'PGX', 'PGZ', 'PH', 'PHAR', 'PHAS', 'PHAT', 'PHB',
           'PHCF', 'PHD', 'PHDG', 'PHG', 'PHGE', 'PHI', 'PHICU', 'PHIO', 'PHK', 'PHM', 'PHO', 'PHR', 'PHT', 'PHUN',
           'PHVS', 'PHX', 'PHYL', 'PHYS', 'PI', 'PICB', 'PICK', 'PICO', 'PID', 'PIE', 'PII', 'PILL', 'PIM', 'PIN',
           'PINC', 'PINE', 'PING', 'PINS', 'PIO', 'PIPP', 'PIPR', 'PIRS', 'PIXY', 'PIZ', 'PJAN', 'PJP', 'PJT', 'PJUL',
           'PJUN', 'PK', 'PKB', 'PKBK', 'PKE', 'PKG', 'PKI', 'PKO', 'PKOH', 'PKW', 'PKX', 'PLAB', 'PLAG', 'PLAN',
           'PLAT', 'PLAY', 'PLBC', 'PLCE', 'PLD', 'PLG', 'PLIN', 'PLL', 'PLM', 'PLMR', 'PLNT', 'PLOW', 'PLPC', 'PLRX',
           'PLSE', 'PLT', 'PLTK', 'PLTM', 'PLTR', 'PLUG', 'PLUS', 'PLW', 'PLX', 'PLXP', 'PLXS', 'PLYA', 'PLYM', 'PM',
           'PMAR', 'PMAY', 'PMBC', 'PMD', 'PME', 'PMF', 'PML', 'PMM', 'PMO', 'PMT', 'PMVC', 'PMVP', 'PMX', 'PNBK',
           'PNC', 'PNF', 'PNFP', 'PNI', 'PNM', 'PNNT', 'PNOV', 'PNQI', 'PNR', 'PNRG', 'PNTG', 'PNW', 'POAI', 'POCT',
           'PODD', 'POLA', 'POOL', 'POR', 'POSH', 'POST', 'POTX', 'POWI', 'POWL', 'POWRU', 'POWW', 'PPA', 'PPBI',
           'PPBT', 'PPC', 'PPD', 'PPG', 'PPH', 'PPIH', 'PPL', 'PPLT', 'PPR', 'PPSI', 'PPT', 'PPTY', 'PQG', 'PRA',
           'PRAA', 'PRAH', 'PRAX', 'PRCH', 'PRDO', 'PREF', 'PRF', 'PRFT', 'PRFX', 'PRFZ', 'PRG', 'PRGO', 'PRGS', 'PRGX',
           'PRI', 'PRIM', 'PRK', 'PRLB', 'PRLD', 'PRMW', 'PRN', 'PRNT', 'PRO', 'PROF', 'PROG', 'PROS', 'PROV', 'PRPB',
           'PRPH', 'PRPL', 'PRPO', 'PRQR', 'PRSP', 'PRT', 'PRTA', 'PRTH', 'PRTK', 'PRTS', 'PRTY', 'PRU', 'PRVB', 'PS',
           'PSA', 'PSAC', 'PSB', 'PSC', 'PSCC', 'PSCD', 'PSCE', 'PSCF', 'PSCH', 'PSCI', 'PSCT', 'PSCU', 'PSEC', 'PSEP',
           'PSET', 'PSF', 'PSHG', 'PSI', 'PSJ', 'PSK', 'PSL', 'PSLV', 'PSMB', 'PSMC', 'PSMG', 'PSMT', 'PSN', 'PSNL',
           'PSO', 'PSP', 'PSQ', 'PSR', 'PST', 'PSTG', 'PSTH', 'PSTI', 'PSTL', 'PSTV', 'PSTX', 'PSX', 'PSXP', 'PT',
           'PTBD', 'PTC', 'PTCT', 'PTE', 'PTEN', 'PTEU', 'PTF', 'PTGX', 'PTH', 'PTICU', 'PTIN', 'PTLC', 'PTMC', 'PTMN',
           'PTN', 'PTNQ', 'PTNR', 'PTON', 'PTPI', 'PTR', 'PTSI', 'PTVCB', 'PTVE', 'PTY', 'PUBM', 'PUI', 'PUK', 'PULM',
           'PULS', 'PUMP', 'PUTW', 'PUYI', 'PVAC', 'PVBC', 'PVG', 'PVH', 'PVI', 'PVL', 'PW', 'PWB', 'PWFL', 'PWOD',
           'PWR', 'PWS', 'PWV', 'PWZ', 'PXD', 'PXE', 'PXF', 'PXH', 'PXI', 'PXJ', 'PXLW', 'PXQ', 'PXS', 'PYN', 'PYPD',
           'PYPE', 'PYPL', 'PYZ', 'PZA', 'PZC', 'PZD', 'PZG', 'PZN', 'PZT', 'PZZA', 'QABA', 'QADA', 'QAI', 'QARP',
           'QAT', 'QCLN', 'QCOM', 'QCRH', 'QD', 'QDEC', 'QDEF', 'QDEL', 'QDF', 'QED', 'QEFA', 'QELL', 'QELLU', 'QEMM',
           'QEP', 'QFIN', 'QGEN', 'QGRO', 'QH', 'QID', 'QINT', 'QIWI', 'QK', 'QLC', 'QLD', 'QLGN', 'QLI', 'QLS', 'QLTA',
           'QLV', 'QLVD', 'QLVE', 'QLYS', 'QMCO', 'QMN', 'QMOM', 'QNST', 'QPX', 'QQEW', 'QQH', 'QQQ', 'QQQE', 'QQQJ',
           'QQQM', 'QQQN', 'QQQX', 'QQXT', 'QRFT', 'QRHC', 'QRTEA', 'QRTEB', 'QRTEP', 'QRVO', 'QS', 'QSR', 'QSY',
           'QTEC', 'QTNT', 'QTRX', 'QTS', 'QTT', 'QTUM', 'QTWO', 'QUAD', 'QUAL', 'QUIK', 'QUMU', 'QUOT', 'QURE', 'QUS',
           'QVAL', 'QYLD', 'QYLG', 'R', 'RA', 'RAACU', 'RAAX', 'RACA', 'RACE', 'RAD', 'RADA', 'RADI', 'RAFE', 'RAIL',
           'RAMP', 'RAND', 'RAPT', 'RARE', 'RAVE', 'RAVI', 'RAVN', 'RBA', 'RBAC', 'RBB', 'RBBN', 'RBC', 'RBCAA', 'RBCN',
           'RBNC', 'RBUS', 'RC', 'RCD', 'RCEL', 'RCG', 'RCHG', 'RCHGU', 'RCI', 'RCII', 'RCKT', 'RCKY', 'RCL', 'RCM',
           'RCMT', 'RCON', 'RCS', 'RCUS', 'RDCM', 'RDFN', 'RDHL', 'RDI', 'RDIV', 'RDN', 'RDNT', 'RDOG', 'RDS-A',
           'RDS-B', 'RDUS', 'RDVT', 'RDVY', 'RDWR', 'RDY', 'RE', 'REAL', 'REDU', 'REED', 'REET', 'REFR', 'REG', 'REGI',
           'REGL', 'REGN', 'REI', 'REK', 'REKR', 'RELL', 'RELX', 'REM', 'REML', 'REMX', 'RENN', 'REPH', 'REPL', 'RES',
           'RESD', 'RESE', 'RESN', 'RESP', 'RETA', 'RETL', 'RETO', 'REV', 'REVG', 'REW', 'REX', 'REXR', 'REYN', 'REZ',
           'REZI', 'RF', 'RFCI', 'RFDA', 'RFDI', 'RFEM', 'RFFC', 'RFG', 'RFI', 'RFIL', 'RFL', 'RFP', 'RFV', 'RGA',
           'RGCO', 'RGEN', 'RGI', 'RGLD', 'RGLS', 'RGNX', 'RGP', 'RGR', 'RGS', 'RGT', 'RH', 'RHE', 'RHI', 'RHP', 'RHS',
           'RIBT', 'RICK', 'RIDE', 'RIG', 'RIGL', 'RIGS', 'RILY', 'RINF', 'RING', 'RIO', 'RIOT', 'RIV', 'RJA', 'RJF',
           'RJI', 'RJN', 'RKDA', 'RKT', 'RL', 'RLAY', 'RLGT', 'RLGY', 'RLH', 'RLI', 'RLJ', 'RLMD', 'RLX', 'RLY', 'RM',
           'RMAX', 'RMBI', 'RMBL', 'RMBS', 'RMCF', 'RMD', 'RMED', 'RMGB', 'RMGBU', 'RMI', 'RMM', 'RMNI', 'RMO', 'RMR',
           'RMRM', 'RMT', 'RMTI', 'RNA', 'RNDM', 'RNDV', 'RNET', 'RNG', 'RNGR', 'RNLC', 'RNLX', 'RNMC', 'RNP', 'RNR',
           'RNRG', 'RNSC', 'RNST', 'RNWK', 'ROAD', 'ROAM', 'ROBO', 'ROBT', 'ROCC', 'ROCCU', 'ROCK', 'RODM', 'ROG',
           'ROIC', 'ROK', 'ROKT', 'ROKU', 'ROL', 'ROLL', 'ROM', 'ROMO', 'ROOF', 'ROOT', 'ROP', 'RORO', 'ROST', 'ROUS',
           'RP', 'RPAI', 'RPAR', 'RPAY', 'RPD', 'RPG', 'RPLA', 'RPM', 'RPRX', 'RPT', 'RPTX', 'RPV', 'RQI', 'RRBI',
           'RRC', 'RRD', 'RRGB', 'RRR', 'RS', 'RSG', 'RSI', 'RSP', 'RSSS', 'RSVA', 'RSVAU', 'RSX', 'RSXJ', 'RTH',
           'RTLR', 'RTM', 'RTPZ', 'RTX', 'RUBY', 'RUHN', 'RUN', 'RUSHA', 'RUSL', 'RUTH', 'RVI', 'RVLV', 'RVMD', 'RVNC',
           'RVNU', 'RVP', 'RVPH', 'RVRS', 'RVSB', 'RVT', 'RWGV', 'RWJ', 'RWK', 'RWL', 'RWLK', 'RWM', 'RWO', 'RWR',
           'RWT', 'RWVG', 'RWX', 'RXD', 'RXI', 'RXL', 'RXN', 'RXT', 'RY', 'RYAAY', 'RYAM', 'RYB', 'RYE', 'RYF', 'RYH',
           'RYI', 'RYJ', 'RYLD', 'RYN', 'RYT', 'RYTM', 'RYU', 'RZG', 'RZLT', 'RZV', 'SA', 'SABR', 'SACH', 'SAFE',
           'SAFM', 'SAFT', 'SAGE', 'SAH', 'SAIA', 'SAIC', 'SAII', 'SAIL', 'SAL', 'SALM', 'SALT', 'SAM', 'SAMG', 'SAN',
           'SANA', 'SAND', 'SANM', 'SANW', 'SAP', 'SAR', 'SASR', 'SATS', 'SAVA', 'SAVE', 'SB', 'SBAC', 'SBB', 'SBBP',
           'SBCF', 'SBE', 'SBFG', 'SBGI', 'SBH', 'SBI', 'SBIO', 'SBLK', 'SBNY', 'SBNYP', 'SBOW', 'SBR', 'SBRA', 'SBS',
           'SBSI', 'SBSW', 'SBT', 'SBTX', 'SBUX', 'SC', 'SCCO', 'SCD', 'SCHA', 'SCHB', 'SCHC', 'SCHD', 'SCHE', 'SCHF',
           'SCHG', 'SCHH', 'SCHI', 'SCHJ', 'SCHK', 'SCHL', 'SCHM', 'SCHN', 'SCHO', 'SCHP', 'SCHQ', 'SCHR', 'SCHV',
           'SCHW', 'SCHX', 'SCHZ', 'SCI', 'SCJ', 'SCKT', 'SCL', 'SCM', 'SCO', 'SCOAU', 'SCOR', 'SCPH', 'SCPL', 'SCPS',
           'SCS', 'SCSC', 'SCU', 'SCVL', 'SCWX', 'SCX', 'SCYX', 'SCZ', 'SD', 'SDC', 'SDD', 'SDEM', 'SDG', 'SDGR',
           'SDIV', 'SDOG', 'SDOW', 'SDP', 'SDPI', 'SDS', 'SDVY', 'SDY', 'SE', 'SEAC', 'SEAH', 'SEAS', 'SEB', 'SECO',
           'SECT', 'SEDG', 'SEE', 'SEED', 'SEEL', 'SEER', 'SEF', 'SEIC', 'SELB', 'SELF', 'SEM', 'SENEA', 'SENS', 'SENT',
           'SESN', 'SF', 'SFBC', 'SFBS', 'SFE', 'SFET', 'SFIG', 'SFIX', 'SFL', 'SFM', 'SFNC', 'SFST', 'SFT', 'SFTW',
           'SFUN', 'SFY', 'SFYF', 'SFYX', 'SG', 'SGA', 'SGAM', 'SGAMU', 'SGBX', 'SGC', 'SGDJ', 'SGDM', 'SGEN', 'SGG',
           'SGH', 'SGLB', 'SGMA', 'SGMO', 'SGMS', 'SGOC', 'SGOL', 'SGOV', 'SGRP', 'SGRY', 'SGTX', 'SGU', 'SH', 'SHACU',
           'SHAG', 'SHAK', 'SHBI', 'SHC', 'SHE', 'SHEN', 'SHG', 'SHI', 'SHIP', 'SHLS', 'SHLX', 'SHM', 'SHO', 'SHOO',
           'SHOP', 'SHSP', 'SHV', 'SHW', 'SHY', 'SHYD', 'SHYF', 'SHYG', 'SHYL', 'SI', 'SIBN', 'SIC', 'SID', 'SIEB',
           'SIEN', 'SIF', 'SIFY', 'SIG', 'SIGA', 'SIGI', 'SIGIP', 'SII', 'SIJ', 'SIL', 'SILC', 'SILJ', 'SILK', 'SILV',
           'SIM', 'SIMO', 'SIMS', 'SINA', 'SINO', 'SINT', 'SIOX', 'SIRI', 'SITC', 'SITE', 'SITM', 'SIVB', 'SIVR', 'SIX',
           'SIXA', 'SIXH', 'SIXL', 'SIXS', 'SIZE', 'SJ', 'SJB', 'SJI', 'SJM', 'SJNK', 'SJR', 'SJT', 'SJW', 'SKF',
           'SKLZ', 'SKM', 'SKOR', 'SKT', 'SKX', 'SKY', 'SKYU', 'SKYW', 'SKYY', 'SLAB', 'SLB', 'SLCA', 'SLCRU', 'SLCT',
           'SLDB', 'SLF', 'SLG', 'SLGG', 'SLGL', 'SLGN', 'SLM', 'SLNO', 'SLP', 'SLQD', 'SLQT', 'SLRC', 'SLRX', 'SLS',
           'SLV', 'SLVO', 'SLVP', 'SLX', 'SLY', 'SLYG', 'SLYV', 'SM', 'SMAR', 'SMB', 'SMBC', 'SMBK', 'SMCI', 'SMCP',
           'SMDD', 'SMDV', 'SMED', 'SMFG', 'SMG', 'SMH', 'SMHB', 'SMIN', 'SMIT', 'SMLF', 'SMLP', 'SMLV', 'SMM', 'SMMD',
           'SMMF', 'SMMT', 'SMMU', 'SMMV', 'SMN', 'SMOG', 'SMP', 'SMPL', 'SMSI', 'SMTC', 'SMTS', 'SMTX', 'SNA', 'SNAP',
           'SNBR', 'SNCA', 'SNCR', 'SND', 'SNDE', 'SNDL', 'SNDR', 'SNDX', 'SNE', 'SNES', 'SNEX', 'SNFCA', 'SNGX',
           'SNLN', 'SNMP', 'SNN', 'SNOA', 'SNOW', 'SNP', 'SNPE', 'SNPR', 'SNPS', 'SNR', 'SNRH', 'SNRHU', 'SNSE', 'SNSR',
           'SNSS', 'SNUG', 'SNV', 'SNX', 'SNY', 'SO', 'SOAC', 'SOCL', 'SOGO', 'SOHO', 'SOHU', 'SOI', 'SOL', 'SOLO',
           'SOLY', 'SON', 'SONA', 'SONM', 'SONN', 'SONO', 'SOR', 'SOS', 'SOVB', 'SOXL', 'SOXS', 'SOXX', 'SOYB', 'SP',
           'SPAB', 'SPAK', 'SPB', 'SPBO', 'SPCB', 'SPCE', 'SPCX', 'SPDN', 'SPDV', 'SPDW', 'SPE', 'SPEM', 'SPEU', 'SPFF',
           'SPFI', 'SPFR', 'SPG', 'SPGI', 'SPGM', 'SPGP', 'SPH', 'SPHB', 'SPHD', 'SPHQ', 'SPHY', 'SPI', 'SPIB', 'SPIP',
           'SPKE', 'SPLB', 'SPLG', 'SPLK', 'SPLP', 'SPLV', 'SPMB', 'SPMD', 'SPMO', 'SPNE', 'SPNS', 'SPNV', 'SPOK',
           'SPOT', 'SPPI', 'SPPP', 'SPR', 'SPRB', 'SPRO', 'SPRQ', 'SPRT', 'SPSB', 'SPSC', 'SPSK', 'SPSM', 'SPT', 'SPTI',
           'SPTL', 'SPTM', 'SPTN', 'SPTS', 'SPUS', 'SPUU', 'SPVM', 'SPVU', 'SPWH', 'SPWR', 'SPXB', 'SPXC', 'SPXE',
           'SPXL', 'SPXS', 'SPXT', 'SPXU', 'SPXX', 'SPXZ', 'SPY', 'SPYD', 'SPYG', 'SPYV', 'SPYX', 'SQ', 'SQBG', 'SQEW',
           'SQFT', 'SQLV', 'SQM', 'SQNS', 'SQQQ', 'SQZ', 'SR', 'SRAC', 'SRACU', 'SRAX', 'SRC', 'SRCE', 'SRCL', 'SRDX',
           'SRE', 'SRET', 'SREV', 'SRG', 'SRGA', 'SRI', 'SRL', 'SRLN', 'SRLP', 'SRNE', 'SRPT', 'SRRA', 'SRRK', 'SRS',
           'SRSAU', 'SRT', 'SRTS', 'SRTY', 'SRV', 'SRVR', 'SSB', 'SSBI', 'SSD', 'SSG', 'SSKN', 'SSL', 'SSNC', 'SSNT',
           'SSO', 'SSP', 'SSPK', 'SSPY', 'SSRM', 'SSSS', 'SSTI', 'SSTK', 'SSUS', 'SSY', 'SSYS', 'ST', 'STAA', 'STAF',
           'STAG', 'STAR', 'STAY', 'STBA', 'STC', 'STCN', 'STE', 'STEP', 'STFC', 'STG', 'STIC', 'STIM', 'STIP', 'STK',
           'STKL', 'STKS', 'STL', 'STLA', 'STLD', 'STLG', 'STM', 'STMP', 'STN', 'STND', 'STNE', 'STNG', 'STOK', 'STON',
           'STOR', 'STOT', 'STPK', 'STPZ', 'STRA', 'STRL', 'STRM', 'STRO', 'STRR', 'STRRP', 'STRS', 'STRT', 'STSA',
           'STT', 'STTK', 'STWD', 'STWO', 'STWOU', 'STX', 'STXB', 'STXS', 'STZ', 'SU', 'SUB', 'SUI', 'SUM', 'SUMO',
           'SUMR', 'SUN', 'SUNS', 'SUNW', 'SUP', 'SUPN', 'SUPV', 'SURF', 'SUSA', 'SUSB', 'SUSC', 'SUSL', 'SUZ', 'SV',
           'SVAC', 'SVACU', 'SVBI', 'SVC', 'SVFA', 'SVFAU', 'SVM', 'SVMK', 'SVOKU', 'SVRA', 'SVSVU', 'SVT', 'SVVC',
           'SVXY', 'SWAN', 'SWAV', 'SWBI', 'SWCH', 'SWI', 'SWIR', 'SWK', 'SWKS', 'SWM', 'SWN', 'SWTX', 'SWX', 'SWZ',
           'SXC', 'SXI', 'SXT', 'SXTC', 'SY', 'SYBT', 'SYBX', 'SYF', 'SYG', 'SYK', 'SYKE', 'SYLD', 'SYN', 'SYNA',
           'SYNC', 'SYNH', 'SYNL', 'SYPR', 'SYRS', 'SYTA', 'SYX', 'SYY', 'SZC', 'SZNE', 'T', 'TA', 'TAC', 'TACA',
           'TACO', 'TACT', 'TAIL', 'TAIT', 'TAK', 'TAL', 'TALO', 'TAN', 'TANH', 'TAOP', 'TAP', 'TARA', 'TARO', 'TARS',
           'TAST', 'TATT', 'TAXF', 'TAYD', 'TBA', 'TBBK', 'TBF', 'TBI', 'TBIO', 'TBK', 'TBLT', 'TBNK', 'TBPH', 'TBT',
           'TBX', 'TC', 'TCBI', 'TCBK', 'TCDA', 'TCF', 'TCFC', 'TCHP', 'TCI', 'TCMD', 'TCOM', 'TCON', 'TCP', 'TCPC',
           'TCRR', 'TCS', 'TCTL', 'TCX', 'TD', 'TDAC', 'TDC', 'TDF', 'TDG', 'TDIV', 'TDOC', 'TDS', 'TDSA', 'TDSB',
           'TDSC', 'TDSD', 'TDSE', 'TDTF', 'TDTT', 'TDV', 'TDW', 'TDY', 'TEAF', 'TEAM', 'TECB', 'TECH', 'TECK', 'TECL',
           'TECS', 'TEDU', 'TEF', 'TEI', 'TEKK', 'TEL', 'TELA', 'TELL', 'TEN', 'TENB', 'TENX', 'TEO', 'TER', 'TERM',
           'TERN', 'TESS', 'TEVA', 'TEX', 'TFC', 'TFFP', 'TFI', 'TFII', 'TFIV', 'TFLO', 'TFSL', 'TFX', 'TG', 'TGA',
           'TGB', 'TGC', 'TGH', 'TGI', 'TGLS', 'TGNA', 'TGP', 'TGS', 'TGT', 'TGTX', 'TH', 'THBR', 'THC', 'THCA', 'THCB',
           'THCX', 'THD', 'THFF', 'THG', 'THM', 'THMAU', 'THMO', 'THO', 'THQ', 'THR', 'THRM', 'THRY', 'THS', 'THTX',
           'THW', 'THY', 'TIG', 'TIGO', 'TIGR', 'TILE', 'TILT', 'TIMB', 'TINV', 'TIP', 'TIPT', 'TIPX', 'TIPZ', 'TIRX',
           'TISI', 'TITN', 'TIXT', 'TJX', 'TK', 'TKAT', 'TKC', 'TKR', 'TLC', 'TLGT', 'TLH', 'TLK', 'TLMD', 'TLND',
           'TLRY', 'TLS', 'TLSA', 'TLT', 'TLTD', 'TLTE', 'TLYS', 'TM', 'TMAT', 'TMBR', 'TMDI', 'TMDX', 'TME', 'TMF',
           'TMFC', 'TMHC', 'TMO', 'TMP', 'TMPMU', 'TMQ', 'TMST', 'TMTS', 'TMUS', 'TMV', 'TMX', 'TNA', 'TNAV', 'TNC',
           'TNDM', 'TNET', 'TNK', 'TNP', 'TNXP', 'TOK', 'TOKE', 'TOL', 'TOLZ', 'TOMZ', 'TOPS', 'TOT', 'TOTL', 'TOUR',
           'TOWN', 'TPAY', 'TPB', 'TPC', 'TPCO', 'TPGY', 'TPH', 'TPHD', 'TPHS', 'TPIC', 'TPIF', 'TPL', 'TPLC', 'TPOR',
           'TPR', 'TPRE', 'TPSC', 'TPTX', 'TPVG', 'TPX', 'TPYP', 'TPZ', 'TQQQ', 'TR', 'TRC', 'TRCH', 'TREB', 'TREC',
           'TREE', 'TREX', 'TRGP', 'TRHC', 'TRI', 'TRIB', 'TRIL', 'TRIN', 'TRIP', 'TRIT', 'TRMB', 'TRMD', 'TRMK',
           'TRMT', 'TRN', 'TRND', 'TRNO', 'TRNS', 'TROW', 'TROX', 'TRP', 'TRQ', 'TRS', 'TRST', 'TRT', 'TRTN', 'TRTX',
           'TRTY', 'TRU', 'TRUE', 'TRUP', 'TRV', 'TRVG', 'TRVI', 'TRVN', 'TRX', 'TRXC', 'TS', 'TSBK', 'TSC', 'TSCO',
           'TSE', 'TSEM', 'TSHA', 'TSI', 'TSIA', 'TSIAU', 'TSLA', 'TSLX', 'TSM', 'TSN', 'TSQ', 'TSRI', 'TT', 'TTAC',
           'TTC', 'TTCF', 'TTD', 'TTEC', 'TTEK', 'TTGT', 'TTI', 'TTM', 'TTMI', 'TTNP', 'TTOO', 'TTP', 'TTT', 'TTWO',
           'TU', 'TUFN', 'TUP', 'TUR', 'TURN', 'TUSK', 'TV', 'TVACU', 'TVE', 'TVTY', 'TW', 'TWI', 'TWIN', 'TWLO', 'TWM',
           'TWN', 'TWNK', 'TWO', 'TWOU', 'TWST', 'TWTR', 'TX', 'TXG', 'TXMD', 'TXN', 'TXRH', 'TXT', 'TY', 'TYD', 'TYG',
           'TYHT', 'TYL', 'TYME', 'TYO', 'TZA', 'TZOO', 'TZPSU', 'U', 'UA', 'UAA', 'UAE', 'UAL', 'UAMY', 'UAN', 'UAPR',
           'UAUG', 'UAVS', 'UBA', 'UBCP', 'UBER', 'UBFO', 'UBOH', 'UBOT', 'UBR', 'UBS', 'UBSI', 'UBT', 'UBX', 'UCBI',
           'UCC', 'UCIB', 'UCL', 'UCO', 'UCON', 'UCTT', 'UCYB', 'UDEC', 'UDN', 'UDOW', 'UDR', 'UE', 'UEC', 'UEIC',
           'UEPS', 'UEVM', 'UFAB', 'UFCS', 'UFEB', 'UFI', 'UFO', 'UFPI', 'UFPT', 'UFS', 'UG', 'UGA', 'UGE', 'UGI',
           'UGL', 'UGP', 'UHAL', 'UHS', 'UHT', 'UI', 'UIHC', 'UIS', 'UITB', 'UIVM', 'UJAN', 'UJB', 'UJUL', 'UJUN', 'UK',
           'UL', 'ULBI', 'ULE', 'ULH', 'ULST', 'ULTA', 'ULTR', 'ULVM', 'UMAR', 'UMBF', 'UMC', 'UMDD', 'UMH', 'UMPQ',
           'UNB', 'UNF', 'UNFI', 'UNG', 'UNH', 'UNIT', 'UNL', 'UNM', 'UNOV', 'UNP', 'UNTY', 'UNVR', 'UOCT', 'UONE',
           'UONEK', 'UPLD', 'UPRO', 'UPS', 'UPST', 'UPV', 'UPW', 'UPWK', 'URA', 'URBN', 'URE', 'URG', 'URGN', 'URI',
           'URNM', 'UROV', 'URTH', 'URTY', 'USA', 'USAC', 'USAI', 'USAK', 'USAP', 'USAS', 'USAT', 'USAU', 'USB', 'USCI',
           'USCR', 'USD', 'USDP', 'USDU', 'USEG', 'USEP', 'USFD', 'USFR', 'USHY', 'USIG', 'USIO', 'USL', 'USLM', 'USM',
           'USMC', 'USMF', 'USMV', 'USNA', 'USO', 'USOI', 'USPH', 'USRT', 'USSG', 'UST', 'USTB', 'USVM', 'USWS', 'USX',
           'USXF', 'UTES', 'UTF', 'UTG', 'UTHR', 'UTI', 'UTL', 'UTMD', 'UTRN', 'UTSI', 'UTSL', 'UTZ', 'UUP', 'UUU',
           'UUUU', 'UVE', 'UVSP', 'UVV', 'UVXY', 'UWM', 'UWMC', 'UXI', 'UXIN', 'UYG', 'UYM', 'V', 'VAC', 'VACQU',
           'VALE', 'VALQ', 'VALU', 'VAPO', 'VAR', 'VAW', 'VB', 'VBF', 'VBFC', 'VBIV', 'VBK', 'VBLT', 'VBND', 'VBR',
           'VBTX', 'VC', 'VCEL', 'VCF', 'VCIF', 'VCIT', 'VCKAU', 'VCLT', 'VCNX', 'VCR', 'VCRA', 'VCSH', 'VCTR', 'VCV',
           'VCVC', 'VCVCU', 'VCYT', 'VDC', 'VDE', 'VEA', 'VEC', 'VECO', 'VEDL', 'VEEV', 'VEGI', 'VEGN', 'VEL', 'VEON',
           'VER', 'VERB', 'VERI', 'VERO', 'VERU', 'VERX', 'VERY', 'VET', 'VEU', 'VFC', 'VFF', 'VFH', 'VFL', 'VFMF',
           'VFMO', 'VFMV', 'VFVA', 'VG', 'VGAC', 'VGI', 'VGIT', 'VGK', 'VGLT', 'VGM', 'VGR', 'VGSH', 'VGT', 'VGZ',
           'VHAQ', 'VHC', 'VHI', 'VHT', 'VIAC', 'VIAO', 'VIAV', 'VICI', 'VICR', 'VIDI', 'VIE', 'VIG', 'VIGI', 'VIH',
           'VINC', 'VINP', 'VIOG', 'VIOO', 'VIOT', 'VIOV', 'VIPS', 'VIR', 'VIRC', 'VIRI', 'VIRS', 'VIRT', 'VIS', 'VISL',
           'VIST', 'VITL', 'VIV', 'VIVE', 'VIVO', 'VIXM', 'VIXY', 'VJET', 'VKI', 'VKQ', 'VKTX', 'VLDR', 'VLGEA', 'VLO',
           'VLRS', 'VLT', 'VLUE', 'VLY', 'VMACU', 'VMAR', 'VMBS', 'VMC', 'VMD', 'VMI', 'VMM', 'VMO', 'VMOT', 'VMW',
           'VNCE', 'VNDA', 'VNE', 'VNET', 'VNLA', 'VNM', 'VNO', 'VNOM', 'VNQ', 'VNQI', 'VNRX', 'VNT', 'VNTR', 'VO',
           'VOC', 'VOD', 'VOE', 'VOLT', 'VONE', 'VONG', 'VONV', 'VOO', 'VOOG', 'VOOV', 'VOR', 'VOSOU', 'VOT', 'VOX',
           'VOXX', 'VOYA', 'VPC', 'VPG', 'VPL', 'VPN', 'VPU', 'VPV', 'VRA', 'VRAI', 'VRAY', 'VRCA', 'VRDN', 'VREX',
           'VRIG', 'VRM', 'VRME', 'VRNA', 'VRNS', 'VRNT', 'VRP', 'VRRM', 'VRS', 'VRSK', 'VRSN', 'VRT', 'VRTS', 'VRTU',
           'VRTV', 'VRTX', 'VS', 'VSAT', 'VSDA', 'VSEC', 'VSGX', 'VSH', 'VSL', 'VSMV', 'VSPRU', 'VSS', 'VST', 'VSTA',
           'VSTM', 'VSTO', 'VT', 'VTA', 'VTAQU', 'VTC', 'VTEB', 'VTGN', 'VTHR', 'VTI', 'VTIP', 'VTIQU', 'VTN', 'VTNR',
           'VTOL', 'VTR', 'VTRS', 'VTRU', 'VTSI', 'VTV', 'VTVT', 'VTWG', 'VTWO', 'VTWV', 'VUG', 'VUSE', 'VUZI', 'VV',
           'VVI', 'VVNT', 'VVOS', 'VVPR', 'VVR', 'VVV', 'VWO', 'VWOB', 'VXF', 'VXRT', 'VXUS', 'VXX', 'VXZ', 'VYGG',
           'VYGR', 'VYM', 'VYMI', 'VYNE', 'VZ', 'W', 'WAB', 'WABC', 'WAFD', 'WAFU', 'WAL', 'WALA', 'WANT', 'WASH',
           'WAT', 'WATT', 'WB', 'WBA', 'WBAI', 'WBIE', 'WBIF', 'WBIG', 'WBII', 'WBIL', 'WBIN', 'WBIT', 'WBIY', 'WBK',
           'WBND', 'WBS', 'WBT', 'WCBR', 'WCC', 'WCLD', 'WCN', 'WD', 'WDAY', 'WDC', 'WDFC', 'WDIV', 'WDR', 'WEA',
           'WEAT', 'WEBL', 'WEBS', 'WEC', 'WEI', 'WELL', 'WEN', 'WERN', 'WES', 'WETF', 'WEX', 'WEYS', 'WF', 'WFC',
           'WFG', 'WFH', 'WFHY', 'WFIG', 'WGO', 'WH', 'WHD', 'WHF', 'WHG', 'WHLM', 'WHLR', 'WHR', 'WIA', 'WIFI', 'WILC',
           'WIMI', 'WINA', 'WING', 'WINT', 'WIP', 'WIRE', 'WISA', 'WISH', 'WIT', 'WIW', 'WIX', 'WIZ', 'WK', 'WKEY',
           'WKHS', 'WLDN', 'WLDR', 'WLFC', 'WLK', 'WLKP', 'WLL', 'WLTW', 'WM', 'WMB', 'WMC', 'WMG', 'WMK', 'WMS', 'WMT',
           'WNC', 'WNEB', 'WNS', 'WNW', 'WOMN', 'WOOD', 'WOOF', 'WOR', 'WORK', 'WORX', 'WOW', 'WPC', 'WPF', 'WPG',
           'WPM', 'WPP', 'WPRT', 'WPS', 'WRAP', 'WRB', 'WRE', 'WRI', 'WRK', 'WRLD', 'WRN', 'WSBC', 'WSBF', 'WSC',
           'WSFS', 'WSM', 'WSO', 'WSR', 'WST', 'WSTG', 'WTBA', 'WTER', 'WTFC', 'WTI', 'WTM', 'WTMF', 'WTRE', 'WTRG',
           'WTRH', 'WTRU', 'WTS', 'WTT', 'WTTR', 'WU', 'WUGI', 'WVE', 'WVFC', 'WVVI', 'WW', 'WWD', 'WWE', 'WWJD', 'WWR',
           'WWW', 'WY', 'WYND', 'WYNN', 'WYY', 'X', 'XAIR', 'XAN', 'XAR', 'XBI', 'XBIO', 'XBIT', 'XBUY', 'XCEM', 'XCUR',
           'XDIV', 'XEC', 'XEL', 'XELA', 'XELB', 'XENE', 'XENT', 'XERS', 'XES', 'XFLT', 'XFOR', 'XGN', 'XHB', 'XHE',
           'XHR', 'XHS', 'XIN', 'XITK', 'XL', 'XLB', 'XLC', 'XLE', 'XLF', 'XLG', 'XLI', 'XLK', 'XLNX', 'XLP', 'XLRE',
           'XLRN', 'XLSR', 'XLU', 'XLV', 'XLY', 'XM', 'XME', 'XMHQ', 'XMLV', 'XMMO', 'XMPT', 'XMVM', 'XNCR', 'XNET',
           'XNTK', 'XOG', 'XOM', 'XOMA', 'XONE', 'XOP', 'XOUT', 'XP', 'XPEL', 'XPER', 'XPEV', 'XPH', 'XPL', 'XPO',
           'XPOA', 'XPP', 'XRAY', 'XRLV', 'XRT', 'XRX', 'XSD', 'XSHD', 'XSHQ', 'XSLV', 'XSMO', 'XSOE', 'XSPA', 'XSVM',
           'XSW', 'XT', 'XTL', 'XTLB', 'XTN', 'XTNT', 'XVZ', 'XWEB', 'XXII', 'XYF', 'XYL', 'XYLD', 'Y', 'YAC', 'YALA',
           'YANG', 'YCBD', 'YCS', 'YELP', 'YETI', 'YEXT', 'YGMZ', 'YI', 'YINN', 'YJ', 'YLD', 'YLDE', 'YMAB', 'YMTX',
           'YNDX', 'YOLO', 'YORW', 'YPF', 'YQ', 'YRCW', 'YRD', 'YSG', 'YTEN', 'YTRA', 'YUM', 'YUMC', 'YVR', 'YXI', 'YY',
           'YYY', 'Z', 'ZAGG', 'ZBH', 'ZBRA', 'ZCMD', 'ZDGE', 'ZEAL', 'ZEN', 'ZEUS', 'ZG', 'ZGBR', 'ZGNX', 'ZI', 'ZIG',
           'ZIM', 'ZION', 'ZIOP', 'ZIXI', 'ZKIN', 'ZLAB', 'ZM', 'ZNGA', 'ZNH', 'ZNTE', 'ZNTEU', 'ZNTL', 'ZOM', 'ZROZ',
           'ZS', 'ZSAN', 'ZSL', 'ZTO', 'ZTR', 'ZTS', 'ZUMZ', 'ZUO', 'ZVO', 'ZWRKU', 'ZYME', 'ZYNE', 'ZYXI']

Skip this also

Skip this also

In [ ]:
tickers_list=tickers[0:500]
for ticker in tickers_list:
  goog_data2 = get_stock(ticker=ticker, start=start_date, end=end_date)
  #goog_data = goog_data2.tail(620)
  goog_data = goog_data2

  try:
    goog_monthly_return=goog_data['close'].pct_change().groupby([goog_data['close'].index.year,goog_data['close'].index.month]).mean()
    goog_monthly_return_list=[]

    for i in  range(len(goog_monthly_return)):
      goog_monthly_return_list.append({'month':goog_monthly_return.index[i][1],'monthly_return'+ticker:goog_monthly_return.iloc[i]})

    goog_monthly_return_list=pd.DataFrame(goog_monthly_return_list,columns=('month','monthly_return'+ticker))
    goog_monthly_return_list
    goog_monthly_return
    goog_monthly_return_list.boxplot(column='monthly_return'+ticker,by='month')
  except:
    continue

